In [1]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
from sklearn.model_selection import train_test_split

import keras
import random
from keras.datasets import mnist
from keras.models import Model
from keras.models import Sequential
from keras import regularizers
from keras import metrics

from keras.layers import Input, Flatten, Dense, Dropout, Lambda, Conv2D, Activation, BatchNormalization, Concatenate
from keras.layers import MaxPooling1D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from keras import backend as K
from matplotlib import pyplot as plt

import pickle
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.externals import joblib
import numpy as np
import os
import glob
from keras.models import model_from_json
import calendar
import time

from sklearn.neighbors import DistanceMetric

Using TensorFlow backend.
/mnt/tmp/anaconda3/envs/sed2019/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
Dir = "/mnt/tmp/"
Train_dir = Dir + "DCASE/Original/train_spec_vgg/"
Test_dir = Dir + "DCASE/Original/validate_spec_vgg/"

index_to_files_dict_train = joblib.load(Dir + 'DCASE/spec_vgg/label_to_files_train.zip')
index_to_files_dict_test = joblib.load(Dir + 'DCASE/spec_vgg/label_to_files_test.zip')

In [3]:
def load_dataset(data_path, ifLoadCoarse):
    filenames = glob.glob(data_path + "*.pkl")
    # count = 0
    features = []
    labels = []
    for i in range(0, len(filenames)):
        spectrogram, vgg_emb, label =  joblib.load(filenames[i])
        #     print(vgg_emb.shape)
        if(ifLoadCoarse == True):
            labels_to_load = label[0:8]   ## load Coarse Level Label
        else:
            labels_to_load = label[8:37]  ## load Fine Level Label

        if(np.count_nonzero(labels_to_load) >= 1):
            features.append(vgg_emb)
            labels.append(labels_to_load)
    return features, labels

In [4]:
def process_VggishFeature(features):
    for i in range(len(features)):
        features[i] = features[i].astype('float32')
        features[i] /= 255
    return features

In [5]:
Train_Features, Train_Labels = load_dataset(Train_dir, True)
Test_Features, Test_Labels = load_dataset(Test_dir, True)

Train_Features = process_VggishFeature(Train_Features)
Test_Features = process_VggishFeature(Test_Features)

In [6]:
num_files = len(Train_Features)
num_time_steps = Train_Features[0].shape[0]
num_features = Train_Features[0].shape[1]
print(num_files)
print(num_time_steps)
print(num_features)

1624
10
128


In [7]:
min_feature = np.ones(128)
max_feature = np.zeros(128)

def getMinimumm():
    num_lines = len(Train_Features)
    for feature_id in range(0, num_features):
        for file_id in range(0, num_files):
#             minElement = np.amin(feature_array_one_file[:, col])
#         print('min', minElement)
            for line_id in range(0, num_time_steps):
                minElement = Train_Features[file_id][line_id][feature_id]
                if(minElement < min_feature[feature_id]):
                    min_feature[feature_id] = minElement
        
def getMaximumm():
    num_lines = len(Train_Features)
    for feature_id in range(0, num_features):
        for file_id in range(0, num_files):
#             minElement = np.amin(feature_array_one_file[:, col])
#         print('min', minElement)
            for line_id in range(0, num_time_steps):
                maxElement = Train_Features[file_id][line_id][feature_id]
#             maxElement = np.amax(feature_array_one_file[:, col])
#         print('max', maxElement)
                if(maxElement > max_feature[feature_id]):
                    max_feature[feature_id] = maxElement 

getMinimumm()
getMaximumm()

In [8]:
def get_NormalizedFeatures(UnNormalizedFeatures):
    Normalized_Features = []
    difference = max_feature - min_feature
    
    num_files = len(UnNormalizedFeatures)
    for file_id in range(0, num_files):
#     one_file_feature = np.zeros(128)
        one_file_feature = np.zeros((10, 128))
    
        for line_id in range(0, num_time_steps):
            for feature_id in range(0, num_features):
                one_file_feature[line_id][feature_id] = (UnNormalizedFeatures[file_id][line_id][feature_id] - min_feature[feature_id])/difference[feature_id]       
        Normalized_Features.append(one_file_feature)    
        
    print(len(Normalized_Features))
    print(Normalized_Features[0].shape)
    return Normalized_Features
# print(Normalized_All_Features_Mean_Std[0])
# print(difference)

In [9]:
Normalized_Train_Features = get_NormalizedFeatures(Train_Features)

1624
(10, 128)


In [10]:
from sklearn.utils import shuffle
# X_train, X_val, Y_train, Y_val = train_test_split(Normalized_Train_Features, Train_Labels, test_size=0.10, random_state=42
                                                  
X_train, Y_train = shuffle(Normalized_Train_Features, Train_Labels, random_state=0)

print(len(X_train))
# print(len(X_val))
print(len(Y_train))
# print(len(Y_val))

X_test = get_NormalizedFeatures(Test_Features)
Y_test = Test_Labels

1624
1624
416
(10, 128)


In [11]:
count = 0
X_test_MultiLabel = []
Y_test_MultiLabel = []

for i in range(0, len(Y_test)):
    if(np.count_nonzero(Y_test[i]) >= 2):
        count = count + 1
        X_test_MultiLabel.append(X_test[i])
        Y_test_MultiLabel.append(Y_test[i])

print(count)
print(len(Train_Labels))

105
1624


In [12]:
count = 0
X_test_SingleLabel = []
Y_test_SingleLabel = []

for i in range(0, len(Y_test)):
    if(np.count_nonzero(Y_test[i]) == 1):
        count = count + 1
        X_test_SingleLabel.append(X_test[i])
        Y_test_SingleLabel.append(Y_test[i])

print(count)
print(len(Train_Labels))

311
1624


In [13]:
time_steps = 10
freq_bins = 128
num_labels = 8

# Hyper parameters
hidden_units = 128
drop_rate = 0.5
batch_size = 32
epochs  = 1

In [14]:
# def create_base_network():  
#     input_layer = Input(shape=(time_steps, freq_bins))

#     a1 = Dense(hidden_units)(input_layer)
#     a1 = BatchNormalization()(a1)
#     a1 = Activation('relu')(a1)
#     a1 = Dropout(drop_rate)(a1)

#     a2 = Dense(hidden_units)(a1)
#     a2 = BatchNormalization()(a2)
#     a2 = Activation('relu')(a2)
#     a2 = Dropout(drop_rate)(a2)

#     a3 = Dense(hidden_units)(a2)
#     a3 = BatchNormalization()(a3)
#     a3 = Activation('relu')(a3)
#     a3 = Dropout(drop_rate)(a3)
#     output_layer = Dense(128, activation='relu')(a3)

#     model = Model(inputs=input_layer, outputs=output_layer)

#     return model

In [15]:
def get_four_conditions(y1, y2):
    y_diff = np.zeros(num_labels * 3)

    for i in range(num_labels):
        if y1[i] == y2[i] and y1[i] == 1:
            y_diff[i*3 + 0] = 1
        elif y1[i] == y2[i] and y1[i] == 0:
            y_diff[i*3 + 1] = 1
        elif y1[i] != y2[i] and y1[i] == 1:
            y_diff[i*3 + 2] = 1
        elif y1[i] != y2[i] and y1[i] == 0:
            y_diff[i*3 + 2] = 1
    return y_diff

In [16]:
def checkIfExist(seen, z1, z2):
#     string_id1 = [(str(z1) + "_" + str(z2))]
#     string_id2 = [(str(z2) + "_" + str(z1))]
    string_id1 = (str(z1) + "_" + str(z2))
    string_id2 = (str(z2) + "_" + str(z1))
    exist = False
    if string_id1 not in seen or string_id2 not in seen:
        seen.add(string_id1)
#         part1 = string_id.split('_')[0]
#         part2 = string_id.split('_')[1]
#         reversed_id = part2 + "_" + part1
#         seen.add(reversed_id)
        seen.add(string_id2)
    else:
        exist = True
    return seen, exist

In [17]:
def create_pairs(x, y, digit_indices_one, digit_indices_zero, offset):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    ids = []
    
    seen = set()
#     n = min([len(digit_indices[d]) for d in range(num_labels)]) - 1
#     print('n is ', n)
    for d in range(num_labels):
        
#         min_index = min(digit_indices_one[d].shape[0], digit_indices_zero[d].shape[0]) - offset
#         max_index = max(digit_indices_one[d].shape[0], digit_indices_zero[d].shape[0])
        
        for k in range(1, offset):
            min_index = min(digit_indices_one[d].shape[0], digit_indices_zero[d].shape[0]) - k
            max_index = max(digit_indices_one[d].shape[0], digit_indices_zero[d].shape[0]) - k
            
            for i in range(min_index):
                second_index = i + k
                
                z1, z2 = digit_indices_one[d][i], digit_indices_one[d][i + k]    ###  same pair  one one 
                seen, exist = checkIfExist(seen, z1, z2)
                if exist == False:
                    pairs += [[x[z1], x[z2]]]
                    y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                    labels += [y_diff]
                    ids += [(str(z1) + "_" + str(z2))]
#                 print("same pair11  ", i , "  ", second_index)
#                 print("same pair11  ", z1 , "  ", z2)
                
                z1, z2 = digit_indices_zero[d][i], digit_indices_zero[d][i + k]  ###  same pair   zero zero 
                seen, exist = checkIfExist(seen, z1, z2)
                if exist == False:
                    pairs += [[x[z1], x[z2]]]                       
                    y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                    labels += [y_diff]
                    ids += [(str(z1) + "_" + str(z2))]
#                 print("same pair00  ", i , "  ", second_index)
#                print("same pair00  ", z1 , "  ", z2)

                z1, z2 = digit_indices_one[d][i], digit_indices_zero[d][i + k]       ###  not same pair  one zero
                seen, exist = checkIfExist(seen, z1, z2)
                if exist == False:
                    pairs += [[x[z1], x[z2]]]
                    y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                    labels += [y_diff]
                    ids += [(str(z1) + "_" + str(z2))]
#                 print("not same pair10  ", i , "  ", second_index)
#                print("not same pair10  ", z1 , "  ", z2)
                
                z1, z2 = digit_indices_one[d][i + k], digit_indices_zero[d][i]  ###  not same pair   zero one 
                seen, exist = checkIfExist(seen, z1, z2)
                if exist == False:
                    pairs += [[x[z1], x[z2]]]
                    y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                    labels += [y_diff]
                    ids += [(str(z1) + "_" + str(z2))]
#                 print("not same pair01  ", second_index , "  ", i)
#                print("not same pair01  ", z1 , "  ", z2)
            
            for i in range(min_index, max_index):
                
                second_index = i + k
                
                if digit_indices_one[d].shape[0] > digit_indices_zero[d].shape[0]:    ##   if there are more 1 than 0 
                    z1, z2 = digit_indices_one[d][i], digit_indices_one[d][i + k]     ###  same pair  one one
                    seen, exist = checkIfExist(seen, z1, z2)
                    if exist == False:
                        pairs += [[x[z1], x[z2]]]                      
                        y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                        labels += [y_diff]
                        ids += [(str(z1) + "_" + str(z2))]
#                     print("same pair11  ", i , "  ", second_index)
#                     print("same pair11  ", z1 , "  ", z2)

                    if ((i - min_index + k) < digit_indices_zero[d].shape[0]):
                        z1, z2 = digit_indices_one[d][i], digit_indices_zero[d][i - min_index + k]  ###  not same pair  one zero 
                        seen, exist = checkIfExist(seen, z1, z2)
                        if exist == False:
                            pairs += [[x[z1], x[z2]]]
                            y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                            labels += [y_diff] 
                            ids += [(str(z1) + "_" + str(z2))]
                            tmp = second_index - min_index
#                     print("not same pair10  ", i , "  ", tmp)
#                     print("not same pair10  ", z1 , "  ", z2)
    
                else:                                                                 ##  if there are more 0 than 1
#                     z1, z2 = digit_indices_one[d][i - min_index], digit_indices_one[d][i - min_index + 2]  ###  same pair 
                    z1, z2 = digit_indices_zero[d][i], digit_indices_zero[d][i + k]     ###  same pair   zero zero 
                    seen, exist = checkIfExist(seen, z1, z2)
                    if exist == False:
                        pairs += [[x[z1], x[z2]]]                     
                        y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                        labels += [y_diff]
                        ids += [(str(z1) + "_" + str(z2))]
#                     print("same pair00  ", i , "  ", second_index)
#                     print("same pair00  ", z1 , "  ", z2)

                    if ((i - min_index + k) < digit_indices_one[d].shape[0]):
                        z1, z2 = digit_indices_one[d][i - min_index + k], digit_indices_zero[d][i]  ###  not same pair one zero
                        seen, exist = checkIfExist(seen, z1, z2)
                        if exist == False:
                            pairs += [[x[z1], x[z2]]]
                            y_diff = get_four_conditions(y[z1], y[z2])
#                 y_diff = get_two_conditions(y[z1], y[z2])
#                 y_diff = get_y_xor(y[z1], y[z2])
#                 y_diff = get_hamming_dis(y[z1], y[z2])
                            labels += [y_diff]
                            ids += [(str(z1) + "_" + str(z2))]
                            tmp = second_index - min_index
#                     print("not same pair10 ", tmp , "  ", i)
#                     print("not same pair10 ", z1 , "  ", z2)
        
    return np.array(pairs), np.array(labels), ids

In [18]:
def get_digit_indices(Feature, Label):
    num_samples = len(Feature)
    print(num_samples)

    digit_indices_one = []
    digit_indices_zero = []
    for i in range(num_labels):
        indexes_for_label_is_one = []
        indexes_for_label_is_zero = []
        for m in range(num_samples):
            if Label[m][i] == 1:
                indexes_for_label_is_one.append(m)
            elif Label[m][i] == 0:
                indexes_for_label_is_zero.append(m)
            else: 
                print("Wrong Label ", Label[m][i])
        
        digit_indices_one.append(np.asarray(indexes_for_label_is_one))   
        digit_indices_zero.append(np.asarray(indexes_for_label_is_zero))  
        
    return digit_indices_one, digit_indices_zero

In [19]:
# def get_digit_indices():
#     num_train_samples = len(X_train)
# #     num_val_samples = len(X_val)
#     num_test_samples = len(X_test)
    
#     num_test_samples_multilabel = len(X_test_MultiLabel)
    
#     print(num_train_samples)
# #     print(num_val_samples)
#     print(num_test_samples)
#     print(num_test_samples_multilabel)

#     digit_indices_train_one = []
#     digit_indices_train_zero = []
#     for i in range(num_labels):
#         indexes_for_label_is_one = []
#         indexes_for_label_is_zero = []
#         for m in range(num_train_samples):
#             if Y_train[m][i] == 1:
#                 indexes_for_label_is_one.append(m)
#             elif Y_train[m][i] == 0:
#                 indexes_for_label_is_zero.append(m)
#             else: 
#                 print("Wrong Label ", Y_train[m][i])
        
#         digit_indices_train_one.append(np.asarray(indexes_for_label_is_one))   
#         digit_indices_train_zero.append(np.asarray(indexes_for_label_is_zero))  
        
# #     digit_indices_val_one = []
# #     digit_indices_val_zero = []
# #     for i in range(num_labels):
# #         indexes_for_label_is_one = []
# #         indexes_for_label_is_zero = []
# #         for m in range(num_val_samples):
# #             if Y_val[m][i] == 1:
# #                 indexes_for_label_is_one.append(m)
# #             elif Y_val[m][i] == 0:
# #                 indexes_for_label_is_zero.append(m)
# #             else: 
# #                 print("Wrong Label ", Y_val[m][i])
        
# #         digit_indices_val_one.append(np.asarray(indexes_for_label_is_one))   
# #         digit_indices_val_zero.append(np.asarray(indexes_for_label_is_zero))  

#     digit_indices_test_one = []
#     digit_indices_test_zero = []
#     for i in range(num_labels):
#         indexes_for_label_is_one = []
#         indexes_for_label_is_zero = []
#         for m in range(num_test_samples):
#             if Y_test[m][i] == 1:
#                 indexes_for_label_is_one.append(m)
#             elif Y_test[m][i] == 0:
#                 indexes_for_label_is_zero.append(m)
#             else: 
#                 print("Wrong Label ", Y_test[m][i])
#         digit_indices_test_one.append(np.asarray(indexes_for_label_is_one))   
#         digit_indices_test_zero.append(np.asarray(indexes_for_label_is_zero))  

        
#     digit_indices_test_multi_label_one = []
#     digit_indices_test_multi_label_zero = []
#     for i in range(num_labels):
#         indexes_for_label_is_one = []
#         indexes_for_label_is_zero = []
#         for m in range(num_test_samples_multilabel):
#             if Y_test_MultiLabel[m][i] == 1:
#                 indexes_for_label_is_one.append(m)
#             elif Y_test_MultiLabel[m][i] == 0:
#                 indexes_for_label_is_zero.append(m)
#             else: 
#                 print("Wrong Label ", Y_test_MultiLabel[m][i])
#         digit_indices_test_multi_label_one.append(np.asarray(indexes_for_label_is_one))   
#         digit_indices_test_multi_label_zero.append(np.asarray(indexes_for_label_is_zero))  
        
    
#     print(len(digit_indices_train_one))
#     print(len(digit_indices_train_zero))

#     print(len(digit_indices_test_one))
#     print(len(digit_indices_test_zero))
    
#     print(len(digit_indices_test_multi_label_one))
#     print(len(digit_indices_test_multi_label_zero))
    
# #     print(len(digit_indices_val_one))
# #     print(len(digit_indices_val_zero))

#     # print(digit_indices_train_one[0].shape)
#     # print(digit_indices_train_zero[0].shape)

#     # print(digit_indices_train_one[1].shape)
#     # print(digit_indices_train_zero[1].shape)
# #     return digit_indices_train_one, digit_indices_train_zero, digit_indices_val_one, digit_indices_val_zero, digit_indices_test_one, digit_indices_test_zero
#     return digit_indices_train_one, digit_indices_train_zero, digit_indices_test_one, digit_indices_test_zero, digit_indices_test_multi_label_one, digit_indices_test_multi_label_zero

In [20]:
# def get_digit_indices():
#     num_train_samples = len(X_train)
#     num_val_samples = len(X_val)
#     num_test_samples = len(X_test)
#     print(num_train_samples)
#     print(num_val_samples)
#     print(num_test_samples)

#     digit_indices_train_one = []
#     digit_indices_train_zero = []
#     for i in range(num_labels):
#         indexes_for_label_is_one = []
#         indexes_for_label_is_zero = []
#         for m in range(num_train_samples):
#             if Y_train[m][i] == 1:
#                 indexes_for_label_is_one.append(m)
#             elif Y_train[m][i] == 0:
#                 indexes_for_label_is_zero.append(m)
#             else: 
#                 print("Wrong Label ", Y_train[m][i])
        
#         digit_indices_train_one.append(np.asarray(indexes_for_label_is_one))   
#         digit_indices_train_zero.append(np.asarray(indexes_for_label_is_zero))  
        
#     digit_indices_val_one = []
#     digit_indices_val_zero = []
#     for i in range(num_labels):
#         indexes_for_label_is_one = []
#         indexes_for_label_is_zero = []
#         for m in range(num_val_samples):
#             if Y_val[m][i] == 1:
#                 indexes_for_label_is_one.append(m)
#             elif Y_val[m][i] == 0:
#                 indexes_for_label_is_zero.append(m)
#             else: 
#                 print("Wrong Label ", Y_val[m][i])
        
#         digit_indices_val_one.append(np.asarray(indexes_for_label_is_one))   
#         digit_indices_val_zero.append(np.asarray(indexes_for_label_is_zero))  

#     digit_indices_test_one = []
#     digit_indices_test_zero = []
#     for i in range(num_labels):
#         indexes_for_label_is_one = []
#         indexes_for_label_is_zero = []
#         for m in range(num_test_samples):
#             if Y_test[m][i] == 1:
#                 indexes_for_label_is_one.append(m)
#             elif Y_test[m][i] == 0:
#                 indexes_for_label_is_zero.append(m)
#             else: 
#                 print("Wrong Label ", Y_test[m][i])
#         digit_indices_test_one.append(np.asarray(indexes_for_label_is_one))   
#         digit_indices_test_zero.append(np.asarray(indexes_for_label_is_zero))  

    
#     print(len(digit_indices_train_one))
#     print(len(digit_indices_train_zero))

#     print(len(digit_indices_test_one))
#     print(len(digit_indices_test_zero))
    
#     print(len(digit_indices_val_one))
#     print(len(digit_indices_val_zero))

#     # print(digit_indices_train_one[0].shape)
#     # print(digit_indices_train_zero[0].shape)

#     # print(digit_indices_train_one[1].shape)
#     # print(digit_indices_train_zero[1].shape)
#     return digit_indices_train_one, digit_indices_train_zero, digit_indices_val_one, digit_indices_val_zero, digit_indices_test_one, digit_indices_test_zero

In [21]:
def remove_Repeated_file(original_pairs, original_labels, all_ids):
    feature_pairs_fliter = []
    label_filter = []
    
    seen = set()
    uniq = []
#     for string_id in all_ids:
    for i in range(0, len(all_ids)):
        string_id1 = all_ids[i]
        reversed_id = string_id1.split('_')[1] + string_id1.split('_')[0]

        if string_id1 not in seen or reversed_id not in seen:
            uniq.append(string_id1)
    #         print(type(x))
    #         print(string_id)
            seen.add(string_id1)
#             part1 = string_id1.split('_')[0]
#             part2 = string_id1.split('_')[1]
#     #         print("part1 ", part1)
#     #         print("part2 ", part2)
#             reversed_id = part2 + part1
    #         print(reversed_id)
            seen.add(reversed_id)
        
            feature_pairs_fliter.append(original_pairs[i])
            label_filter.append(original_labels[i])
    
    feature_pairs_fliter = np.asarray(feature_pairs_fliter)
    label_filter = np.asarray(label_filter)
    return feature_pairs_fliter, label_filter, uniq

In [22]:
# digit_indices_train_one, digit_indices_train_zero, digit_indices_val_one, digit_indices_val_zero, digit_indices_test_one, digit_indices_test_zero = get_digit_indices()

# digit_indices_train_one, digit_indices_train_zero, digit_indices_test_one, digit_indices_test_zero, digit_indices_test_multi_label_one, digit_indices_test_multi_label_zero = get_digit_indices()

digit_indices_train_one, digit_indices_train_zero = get_digit_indices(X_train, Y_train)
digit_indices_test_one, digit_indices_test_zero = get_digit_indices(X_test, Y_test)
digit_indices_test_multi_label_one, digit_indices_test_multi_label_zero = get_digit_indices(X_test_MultiLabel, Y_test_MultiLabel)
digit_indices_test_single_label_one, digit_indices_test_single_label_zero = get_digit_indices(X_test_SingleLabel, Y_test_SingleLabel)

1624
416
105
311


In [23]:
# print(len(digit_indices_train_one))
# print(digit_indices_train_one[0].shape)
# print(digit_indices_train_one[1].shape)

# print(len(digit_indices_train_zero))
# print(digit_indices_train_zero[0].shape)
# print(digit_indices_train_zero[1].shape)

# print(len(digit_indices_val_one))
# print(digit_indices_val_one[0].shape)
# print(digit_indices_val_one[1].shape)

# print(len(digit_indices_val_zero))
# print(digit_indices_val_zero[0].shape)
# print(digit_indices_val_zero[1].shape)

# print(len(digit_indices_test_one))
# print(digit_indices_test_one[0].shape)
# print(digit_indices_test_one[1].shape)

# print(len(digit_indices_test_zero))
# print(digit_indices_test_one[0].shape)
# print(digit_indices_test_one[1].shape)

In [24]:
tr_pairs, tr_y, tr_pairs_ids = create_pairs(X_train, Y_train, digit_indices_train_one, digit_indices_train_zero, 30)
print(tr_pairs.shape)
print(tr_y.shape)
print(len(tr_pairs_ids))

(246071, 2, 10, 128)
(246071, 24)
246071


In [25]:
# tv_pairs, tv_y, tv_pairs_ids = create_pairs(X_val, Y_val, digit_indices_val_one, digit_indices_val_zero, 30)
# print(tv_pairs.shape)
# print(tv_y.shape)
# print(len(tv_pairs_ids))

In [26]:
te_pairs, te_y, te_pairs_ids = create_pairs(X_test, Y_test, digit_indices_test_one, digit_indices_test_zero, 30)
print(te_pairs.shape)
print(te_y.shape)
print(len(te_pairs_ids))

(39458, 2, 10, 128)
(39458, 24)
39458


In [27]:
te_pairs_multi_label, te_y_multi_label, te_pairs_ids_multi_label = create_pairs(X_test_MultiLabel, Y_test_MultiLabel, digit_indices_test_multi_label_one, digit_indices_test_multi_label_zero, 20)
print(te_pairs_multi_label.shape)
print(te_y_multi_label.shape)
print(len(te_pairs_ids_multi_label))

(4342, 2, 10, 128)
(4342, 24)
4342


In [28]:
te_pairs_single_label, te_y_single_label, te_pairs_ids_single_label = create_pairs(X_test_SingleLabel, Y_test_SingleLabel, digit_indices_test_single_label_one, digit_indices_test_single_label_zero, 20)
print(te_pairs_single_label.shape)
print(te_y_single_label.shape)
print(len(te_pairs_ids_single_label))

(21072, 2, 10, 128)
(21072, 24)
21072


In [29]:
# tr_y_filter = np.expand_dims(tr_y_filter, axis=1)
# print(tr_y_filter.shape)

# te_y_filter = np.expand_dims(te_y_filter, axis=1)
# print(te_y_filter.shape)

In [30]:
# num_train_samples = 732913
# num_test_samples = 100421
# num_train_samples = 1723990
# num_test_samples = 394150
# num_train_samples = 180491
# num_test_samples = 30869
num_train_samples = len(tr_pairs_ids)
# num_val_samples = len(tv_pairs_ids)
num_test_samples = len(te_pairs_ids)
num_test_samples_multi_label = len(te_pairs_multi_label)
num_test_samples_single_label = len(te_pairs_single_label)


# tr_y_filter = tr_y_filter.reshape(num_train_samples, 8, 3)
# te_y_filter = te_y_filter.reshape(num_test_samples, 8, 3)
tr_y = tr_y.reshape(num_train_samples, num_labels, 3)
# tv_y = tv_y.reshape(num_val_samples, 8, 3)
te_y = te_y.reshape(num_test_samples, num_labels, 3)

te_y_multi_label = te_y_multi_label.reshape(num_test_samples_multi_label, num_labels, 3)

te_y_single_label = te_y_single_label.reshape(num_test_samples_single_label, num_labels, 3)

In [31]:
train_out_1 = tr_y[:num_train_samples, 0, :3]
train_out_2 = tr_y[:num_train_samples, 1, :3]
train_out_3 = tr_y[:num_train_samples, 2, :3]
train_out_4 = tr_y[:num_train_samples, 3, :3]
train_out_5 = tr_y[:num_train_samples, 4, :3]
train_out_6 = tr_y[:num_train_samples, 5, :3]
train_out_7 = tr_y[:num_train_samples, 6, :3]
train_out_8 = tr_y[:num_train_samples, 7, :3]

print(train_out_1.shape)
print(train_out_2.shape)
print(train_out_3.shape)
print(train_out_4.shape)
print(train_out_5.shape)
print(train_out_6.shape)
print(train_out_7.shape)
print(train_out_8.shape)

(246071, 3)
(246071, 3)
(246071, 3)
(246071, 3)
(246071, 3)
(246071, 3)
(246071, 3)
(246071, 3)


In [32]:
# val_out_1 = tv_y[:num_train_samples, 0, :3]
# val_out_2 = tv_y[:num_train_samples, 1, :3]
# val_out_3 = tv_y[:num_train_samples, 2, :3]
# val_out_4 = tv_y[:num_train_samples, 3, :3]
# val_out_5 = tv_y[:num_train_samples, 4, :3]
# val_out_6 = tv_y[:num_train_samples, 5, :3]
# val_out_7 = tv_y[:num_train_samples, 6, :3]
# val_out_8 = tv_y[:num_train_samples, 7, :3]

# print(val_out_1.shape)
# print(val_out_2.shape)
# print(val_out_3.shape)
# print(val_out_4.shape)
# print(val_out_5.shape)
# print(val_out_6.shape)
# print(val_out_7.shape)
# print(val_out_8.shape)

In [33]:
test_out_1 = te_y[:num_test_samples, 0, :3]
test_out_2 = te_y[:num_test_samples, 1, :3]
test_out_3 = te_y[:num_test_samples, 2, :3]
test_out_4 = te_y[:num_test_samples, 3, :3]
test_out_5 = te_y[:num_test_samples, 4, :3]
test_out_6 = te_y[:num_test_samples, 5, :3]
test_out_7 = te_y[:num_test_samples, 6, :3]
test_out_8 = te_y[:num_test_samples, 7, :3]

print(test_out_1.shape)
print(test_out_2.shape)
print(test_out_3.shape)
print(test_out_4.shape)
print(test_out_5.shape)
print(test_out_6.shape)
print(test_out_7.shape)
print(test_out_8.shape)

(39458, 3)
(39458, 3)
(39458, 3)
(39458, 3)
(39458, 3)
(39458, 3)
(39458, 3)
(39458, 3)


In [34]:
test_multi_label_out_1 = te_y_multi_label[:num_test_samples_multi_label, 0, :3]
test_multi_label_out_2 = te_y_multi_label[:num_test_samples_multi_label, 1, :3]
test_multi_label_out_3 = te_y_multi_label[:num_test_samples_multi_label, 2, :3]
test_multi_label_out_4 = te_y_multi_label[:num_test_samples_multi_label, 3, :3]
test_multi_label_out_5 = te_y_multi_label[:num_test_samples_multi_label, 4, :3]
test_multi_label_out_6 = te_y_multi_label[:num_test_samples_multi_label, 5, :3]
test_multi_label_out_7 = te_y_multi_label[:num_test_samples_multi_label, 6, :3]
test_multi_label_out_8 = te_y_multi_label[:num_test_samples_multi_label, 7, :3]

print(test_multi_label_out_1.shape)
print(test_multi_label_out_2.shape)
print(test_multi_label_out_3.shape)
print(test_multi_label_out_4.shape)
print(test_multi_label_out_5.shape)
print(test_multi_label_out_6.shape)
print(test_multi_label_out_7.shape)
print(test_multi_label_out_8.shape)

(4342, 3)
(4342, 3)
(4342, 3)
(4342, 3)
(4342, 3)
(4342, 3)
(4342, 3)
(4342, 3)


In [35]:
test_single_label_out_1 = te_y_single_label[:num_test_samples_single_label, 0, :3]
test_single_label_out_2 = te_y_single_label[:num_test_samples_single_label, 1, :3]
test_single_label_out_3 = te_y_single_label[:num_test_samples_single_label, 2, :3]
test_single_label_out_4 = te_y_single_label[:num_test_samples_single_label, 3, :3]
test_single_label_out_5 = te_y_single_label[:num_test_samples_single_label, 4, :3]
test_single_label_out_6 = te_y_single_label[:num_test_samples_single_label, 5, :3]
test_single_label_out_7 = te_y_single_label[:num_test_samples_single_label, 6, :3]
test_single_label_out_8 = te_y_single_label[:num_test_samples_single_label, 7, :3]

print(test_single_label_out_1.shape)
print(test_single_label_out_2.shape)
print(test_single_label_out_3.shape)
print(test_single_label_out_4.shape)
print(test_single_label_out_5.shape)
print(test_single_label_out_6.shape)
print(test_single_label_out_7.shape)
print(test_single_label_out_8.shape)

(21072, 3)
(21072, 3)
(21072, 3)
(21072, 3)
(21072, 3)
(21072, 3)
(21072, 3)
(21072, 3)


In [36]:
def create_base_network():  
    input_layer = Input(shape=(time_steps, freq_bins))

    a1 = Dense(hidden_units)(input_layer)
    a1 = BatchNormalization()(a1)
    a1 = Activation('relu')(a1)
    a1 = Dropout(drop_rate)(a1)

    a2 = Dense(hidden_units)(a1)
    a2 = BatchNormalization()(a2)
    a2 = Activation('relu')(a2)
    a2 = Dropout(drop_rate)(a2)

    a3 = Dense(hidden_units)(a2)
    a3 = BatchNormalization()(a3)
    a3 = Activation('relu')(a3)
    a3 = Dropout(drop_rate)(a3)
    output_layer = Dense(128, activation='relu')(a3)

    model = Model(inputs=input_layer, outputs=output_layer)

    return model

In [37]:
def attention_pooling(inputs, **kwargs):
    [out, att] = inputs

    epsilon = 1e-7
    att = K.clip(att, epsilon, 1. - epsilon)
    normalized_att = att / K.sum(att, axis=1)[:, None, :]

    return K.sum(out * normalized_att, axis=1)

In [38]:
def pooling_shape(input_shape):

    if isinstance(input_shape, list):
        (sample_num, time_steps, freq_bins) = input_shape[0]

    else:
        (sample_num, time_steps, freq_bins) = input_shape

    return (sample_num, freq_bins)

In [39]:
def build_model():
   # network definition
    base_network = create_base_network()

    input_a = Input(shape=(time_steps, freq_bins))
    input_b = Input(shape=(time_steps, freq_bins))

    # because we re-use the same instance `base_network`,
    # the weights of the network
    # will be shared across the two branches
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)

    ### trying to add attention module before the comparision but not sure whether it will work, since the Siamese Network Will o longer be actual Siamese  
    ###### This is not actually distance (it didn't sum up each element) so that the output layer is not 1 node 
#     L1_layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
#     L1_distance = L1_layer([processed_a, processed_b])
    
    ConcatLayer = keras.layers.concatenate([processed_a, processed_b], axis=-1)

#     fc1 = Dense(256)(L1_distance)
    fc1 = Dense(256)(ConcatLayer)
    fc1 = BatchNormalization()(fc1)
    fc1 = Dropout(0.2)(fc1)
    fc1 = Activation("relu")(fc1)

    fc2 = Dense(128)(fc1)
    fc2 = BatchNormalization()(fc2)
    fc2 = Activation('relu')(fc2)

    cla_1 =  Dense(3, activation='softmax')(fc2)
    att_1 = Dense(3, activation='softmax')(fc2)
    output_1 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_1, att_1])

    cla_2 =  Dense(3, activation='softmax')(fc2)
    att_2 = Dense(3, activation='softmax')(fc2)
    output_2 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_2, att_2])

    cla_3 =  Dense(3, activation='softmax')(fc2)
    att_3 = Dense(3, activation='softmax')(fc2)
    output_3 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_3, att_3])

    cla_4 =  Dense(3, activation='softmax')(fc2)
    att_4 = Dense(3, activation='softmax')(fc2)
    output_4 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_4, att_4])

    cla_5 =  Dense(3, activation='softmax')(fc2)
    att_5 = Dense(3, activation='softmax')(fc2)
    output_5 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_5, att_5])

    cla_6 =  Dense(3, activation='softmax')(fc2)
    att_6 = Dense(3, activation='softmax')(fc2)
    output_6 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_6, att_6])

    cla_7 =  Dense(3, activation='softmax')(fc2)
    att_7 = Dense(3, activation='softmax')(fc2)
    output_7 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_7, att_7])

    cla_8 =  Dense(3, activation='softmax')(fc2)
    att_8 = Dense(3, activation='softmax')(fc2)
    output_8 = Lambda(attention_pooling, output_shape=pooling_shape)([cla_8, att_8])


    model = Model(inputs=[input_a, input_b], outputs=[output_1, output_2, output_3, output_4,
                                                  output_5, output_6, output_7, output_8])

#     model.summary()
    print()
#     base_network.summary()
    return model

In [40]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
#         print("true_positives ",K.eval(true_positives))
#         print("possible_positives ",K.eval(possible_positives))
#         print("recall         ",K.eval(recall))
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        
#         print("true_positives ",K.eval(true_positives))
#         print("predicted_positives ",K.eval(predicted_positives))
#         print("precision      ",K.eval(precision))
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    f_score = 2*((precision*recall)/(precision+recall+K.epsilon()))
    
#     print("precision     ",K.eval(precision))
#     print("recall        ",K.eval(recall))
#     print("f_score       ",K.eval(f_score))
    return f_score

In [41]:
def train_model(model):
    epochs = 3
    # train
    rms = RMSprop()
    multiple_losses = ['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy', 
                   'categorical_crossentropy', 'categorical_crossentropy','categorical_crossentropy', 
                   'categorical_crossentropy','categorical_crossentropy'
                  ]
# model.compile(loss= multiple_losses, metrics=['binary_accuracy', metrics.categorical_accuracy], optimizer=rms)
# model.compile(loss= multiple_losses, metrics=[metrics.categorical_accuracy,
#                                               f1_m, 
#                                               precision_m, 
#                                               recall_m], optimizer=rms)
    model.compile(loss= multiple_losses, metrics=[metrics.categorical_accuracy, f1_m,precision_m, recall_m], optimizer=rms)

    history = model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], [train_out_1,train_out_2,train_out_3,train_out_4,
                                                       train_out_5,train_out_6,train_out_7,train_out_8],
                        batch_size=128,
                        epochs=epochs)
    return model


In [42]:
def save_model(model, timestamp, Save_Model_Dir):
#     Save_Model_Dir = Dir + "DCASE/saved_models/"
    # serialize model to JSON
    model_json = model.to_json()
    
    json_dir = Save_Model_Dir + "model_" + str(timestamp) + ".json"
    with open(json_dir, "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    h5_dir = Save_Model_Dir + "model_" + str(timestamp) + ".h5"
    model.save_weights(h5_dir)
    print("Saved model to disk")

In [43]:
def load_model(timestamp, Save_Model_Dir):
    # load json and create model
    json_dir = Save_Model_Dir + "model_" + str(timestamp) + ".json"
    json_file = open(json_dir, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    h5_dir = Save_Model_Dir + "model_" + str(timestamp) + ".h5"
    loaded_model.load_weights(h5_dir)
    print("Loaded model from disk")
    return loaded_model

In [44]:
Database_X = Normalized_Train_Features
Database_Y = Train_Labels
size_of_database = len(Database_Y)

In [45]:
# Threshold = 6

In [46]:
def groundtruth_indicateor_function(groundtruth_item, Threshold):
    actual_similarity = groundtruth_item.get("similarity")
    if(actual_similarity >= Threshold):
#         print(" actual_similarity  ", actual_similarity, "  True")
        return True
    elif (actual_similarity < Threshold):
#         print(" actual_similarity  ", actual_similarity, "  False")
        return False
    else: 
#         print("Wrong ")
        return False

In [56]:
def groundtruth_Average_Precision_at_Top_K(ranked_groundtruth_list, Top_K, Threshold):
    
    average_precision = 0 
    num_positive_hits = 0
    for i in range(0, Top_K):
#     for i in range(size_of_database - Top_K, size_of_database):
        if(groundtruth_indicateor_function(ranked_groundtruth_list[i], Threshold) == True):
            num_positive_hits = num_positive_hits + 1
            average_precision = average_precision + num_positive_hits/(i+1)

    if (num_positive_hits != 0):
        average_precision = average_precision/num_positive_hits
        
    return average_precision

In [57]:
def indicateor_function(predicted_dic, groundtruth_list, Threshold):
    index = predicted_dic.get("index")
    for i in range(len(groundtruth_list)):
        if(groundtruth_list[i].get("index") == index):
            actual_similarity = groundtruth_list[i].get("similarity")
            break
    if(actual_similarity >= Threshold):
#         print(" actual_similarity  ", actual_similarity, "  True")
        return True
    elif (actual_similarity < Threshold):
#         print(" actual_similarity  ", actual_similarity, "  False")
        return False
    else: 
#         print("Wrong ")
        return False

In [58]:
def Average_Precision_at_Top_K(ranked_list, groundtruth_list, Top_K, Threshold):
    
    average_precision = 0 
    num_positive_hits = 0
    for i in range(0, Top_K):
#     for i in range(size_of_database - Top_K, size_of_database):
        if(indicateor_function(ranked_list[i], groundtruth_list, Threshold) == True):
            num_positive_hits = num_positive_hits + 1
            average_precision = average_precision + num_positive_hits/(i+1)

    if (num_positive_hits != 0):
        average_precision = average_precision/num_positive_hits
        
    return average_precision

In [59]:
def Baseline_Average_Precision_at_Top_K(groundtruth_list, Top_K, Threshold):
    
    average_precision = 0 
    num_positive_hits = 0
    
    random_top_k = random.sample(groundtruth_list, Top_K)
    
    for i in range(0, Top_K):
        actual_similarity = random_top_k[i].get("similarity")
        if(actual_similarity >= Threshold):
            num_positive_hits = num_positive_hits + 1
            average_precision = average_precision + num_positive_hits/(i+1)
    if (num_positive_hits != 0):
        average_precision = average_precision/num_positive_hits    
        
    return average_precision

In [60]:
def Compute_Mean_Average_Precision(Threshold, model_load, Test_dataset, Test_dataset_label):
    
    num_query_samples = len(Test_dataset)
    
    map_1 = 0
    map_5 = 0
    map_10 = 0
    map_30 = 0
    map_50 = 0
    map_100 = 0
#     map_database_size = 0
    
    base_map_1 = 0
    base_map_5 = 0
    base_map_10 = 0
    base_map_30 = 0
    base_map_50 = 0
    base_map_100 = 0
#     base_map_database_size = 0
    
    upperbound_map_1 = 0
    upperbound_map_5 = 0
    upperbound_map_10 = 0
    upperbound_map_30 = 0
    upperbound_map_50 = 0
    upperbound_map_100 = 0
#     upperbound_map_database_size = 0

    for i in range (0, num_query_samples):
    
        predicted_ranked_list = []
        groundtruth_list = []
        for k in range(len(Database_X)):
#         print(X_test[i].shape)
            input_1 = np.expand_dims(Test_dataset[i], axis=0)
            input_2 = np.expand_dims(Database_X[k], axis=0)
            pair = [input_1, input_2]
            output = model_load.predict(pair)
        
            ## get one hot presentation of predicted results 
            prediced_results = []
            for n in range (0, num_labels):
                predicted = np.zeros(3)
                index = np.argmax(output[n])
                predicted[index] = 1
                prediced_results.append(predicted)
        
            ### compute the level of similarity based on predicted result
            predicted_similarity = 0
            for n in range (0, num_labels):
                predicted_similarity = predicted_similarity + prediced_results[n][0] + prediced_results[n][1]
#         print("predict similarity " , predicted_similarity)
            thisdict = {"index": k, "similarity": predicted_similarity}
            predicted_ranked_list.append(thisdict)
            
            ### compute the level of similarity of of groundtruth annotation 
            ground_truth = get_four_conditions(Test_dataset_label[i], Database_Y[k])
            ground_truth = ground_truth.reshape(8, 3)
        
#         print("Query ", i, "  ", "Database ", k)
#         print("predicted ")
#         print(np.array(prediced_results))
#         print("ground_tr ")
#         print(ground_truth)  
#         print()
        
            actual_similarity = 0
            for n in range (0, num_labels):
                actual_similarity = actual_similarity + ground_truth[n][0] + ground_truth[n][1]
#         print("ground_truth similarity " , actual_similarity) 
            thisdict = {"index": k, "similarity": actual_similarity}
            groundtruth_list.append(thisdict)
        
#         print("Query ", i)
#     predicted_ranked_list = RankBasedonSimilarity(predicted_ranked_list)
        Ranked_List = sorted(predicted_ranked_list, key = lambda i: i['similarity'], reverse=True)
    
        UpperBound = sorted(groundtruth_list, key = lambda i: i['similarity'], reverse=True)
        
        
        average_precision_1 = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, 1, Threshold)
        map_1 = map_1 + average_precision_1
        
        average_precision_5 = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, 5, Threshold)
        map_5 = map_5 + average_precision_5
        
        average_precision_10 = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, 10, Threshold)
        map_10 = map_10 + average_precision_10
        
        average_precision_30 = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, 30, Threshold)
        map_30 = map_30 + average_precision_30
        
        average_precision_50 = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, 50, Threshold)
        map_50 = map_50 + average_precision_50
        
        average_precision_100 = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, 100, Threshold)
        map_100 = map_100 + average_precision_100
        
#         average_precision_database_size = Average_Precision_at_Top_K(Ranked_List, groundtruth_list, size_of_database, Threshold)
#         map_database_size = map_database_size + average_precision_database_size
        
        
 
        upperbound_average_precision_1 = groundtruth_Average_Precision_at_Top_K(UpperBound, 1, Threshold)
        upperbound_map_1 = upperbound_map_1 + upperbound_average_precision_1
        
        upperbound_average_precision_5 = groundtruth_Average_Precision_at_Top_K(UpperBound, 5, Threshold)
        upperbound_map_5 = upperbound_map_5 + upperbound_average_precision_5
        
        upperbound_average_precision_10 = groundtruth_Average_Precision_at_Top_K(UpperBound, 10, Threshold)
        upperbound_map_10 = upperbound_map_10 + upperbound_average_precision_10
        
        upperbound_average_precision_30 = groundtruth_Average_Precision_at_Top_K(UpperBound, 30, Threshold)
        upperbound_map_30 = upperbound_map_30 + upperbound_average_precision_30
        
        upperbound_average_precision_50 = groundtruth_Average_Precision_at_Top_K(UpperBound, 50, Threshold)
        upperbound_map_50 = upperbound_map_50 + upperbound_average_precision_50
        
        upperbound_average_precision_100 = groundtruth_Average_Precision_at_Top_K(UpperBound, 100, Threshold)
        upperbound_map_100 = upperbound_map_100 + upperbound_average_precision_100
        
        
        
        base_average_precision_1 = Baseline_Average_Precision_at_Top_K(groundtruth_list, 1, Threshold)
        base_map_1 = base_map_1 + base_average_precision_1
        
        base_average_precision_5 = Baseline_Average_Precision_at_Top_K(groundtruth_list, 5, Threshold)
        base_map_5 = base_map_5 + base_average_precision_5
        
        base_average_precision_10 = Baseline_Average_Precision_at_Top_K(groundtruth_list, 10, Threshold)
        base_map_10 = base_map_10 + base_average_precision_10
        
        base_average_precision_30 = Baseline_Average_Precision_at_Top_K(groundtruth_list, 30, Threshold)
        base_map_30 = base_map_30 + base_average_precision_30
        
        base_average_precision_50 = Baseline_Average_Precision_at_Top_K(groundtruth_list, 50, Threshold)
        base_map_50 = base_map_50 + base_average_precision_50
        
        base_average_precision_100 = Baseline_Average_Precision_at_Top_K(groundtruth_list, 100, Threshold)
        base_map_100 = base_map_100 + base_average_precision_100
        
#         base_average_precision_database_size = Baseline_Average_Precision_at_Top_K(groundtruth_list, size_of_database, Threshold)
#         base_map_database_size = base_map_database_size + base_average_precision_database_size
        
#         print("hello")
#         print(base_map_10)
        
        
    map_1 = map_1/num_query_samples
    map_5 = map_5/num_query_samples
    map_10 = map_10/num_query_samples
    map_30 = map_30/num_query_samples
    map_50 = map_50/num_query_samples
    map_100 = map_100/num_query_samples
#     map_database_size = map_database_size/num_query_samples
    
    upperbound_map_1 = upperbound_map_1/num_query_samples
    upperbound_map_5 = upperbound_map_5/num_query_samples
    upperbound_map_10 = upperbound_map_10/num_query_samples
    upperbound_map_30 = upperbound_map_30/num_query_samples
    upperbound_map_50 = upperbound_map_50/num_query_samples
    upperbound_map_100 = upperbound_map_100/num_query_samples

    base_map_1 = base_map_1/num_query_samples
    base_map_5 = base_map_5/num_query_samples
    base_map_10 = base_map_10/num_query_samples
    base_map_30 = base_map_30/num_query_samples
    base_map_50 = base_map_50/num_query_samples
    base_map_100 = base_map_100/num_query_samples
#     base_map_database_size = base_map_database_size/num_query_samples
    
#     map_results = [map_1, map_5, map_10, map_30, map_50, map_100, map_database_size]
#     base_results = [base_map_1, base_map_5, base_map_10, base_map_30, base_map_50, base_map_100, base_map_database_size]
    map_results = [map_1, map_5, map_10, map_30, map_50, map_100]
    base_results = [base_map_1, base_map_5, base_map_10, base_map_30, base_map_50, base_map_100]
    upper_results = [upperbound_map_1, upperbound_map_5, upperbound_map_10, upperbound_map_30, upperbound_map_50, upperbound_map_100]
    
    return map_results, base_results, upper_results

In [61]:
def Categorical_Accuracy_Evaluation(model, paires, out):
#     te_pairs.shape
    num_test_samples = len(paires)

    out_1 = out[0]
    out_2 = out[1]
    out_3 = out[2]
    out_4 = out[3]
    out_5 = out[4]
    out_6 = out[5]
    out_7 = out[6]
    out_8 = out[7]

    input_1 = te_pairs[:num_test_samples, 0, :time_steps, :128]
    input_2 = te_pairs[:num_test_samples, 1, :time_steps, :128]
    Performance_tmp = model.test_on_batch([input_1, input_2], [out_1, out_2, out_3,out_4,out_5,out_6,out_7,out_8])
    
#     print(len(Performance_tmp))
    Performance = Performance_tmp[9:]
#     print(len(Performance))
    acc_summary = []
    f1_summary = []
    precision_summary = []
    recall_summary = []
    for i in range(0, num_labels):
    #     print(i * 4 + 0)
    #     print(Performance[i * 4 + 0])
        acc_summary.append(Performance[i * 4 + 0])
        f1_summary.append(Performance[i * 4 + 1])
        precision_summary.append(Performance[i * 4 + 2])
        recall_summary.append(Performance[i * 4 + 3])
    
    print(acc_summary)
#     print(precision_summary)
#     print(recall_summary )
#     print(f1_summary)
    

In [62]:
print(len(te_pairs_multi_label))

4342


In [63]:
test_out = [test_out_1, test_out_2, test_out_3, test_out_4, test_out_5, test_out_6, test_out_7, test_out_8]
test_multi_label_out = [test_multi_label_out_1, test_multi_label_out_2, test_multi_label_out_3, test_multi_label_out_4, test_multi_label_out_5, test_multi_label_out_6, test_multi_label_out_7, test_multi_label_out_8]
test_single_label_out = [test_single_label_out_1, test_single_label_out_2, test_single_label_out_3, test_single_label_out_4, test_single_label_out_5, test_single_label_out_6, test_single_label_out_7, test_single_label_out_8]

Match_8_MultiLabel = []
Match_8_MultiLabel_Baseline = []
Match_8_MultiLabel_Upperbound = []
Match_8_SingleLabel = []
Match_8_SingleLabel_Baseline = []
Match_8_SingleLabel_Upperbound = []
Match_8_MultiLabel_and_SingleLabel = []
Match_8_MultiLabel_and_SingleLabel_Baseline = []
Match_8_MultiLabel_and_SingleLabel_Upperbound = []

Match_7_MultiLabel = []
Match_7_MultiLabel_Baseline = []
Match_7_MultiLabel_Upperbound = []
Match_7_SingleLabel = []
Match_7_SingleLabel_Baseline = []
Match_7_SingleLabel_Upperbound = []
Match_7_MultiLabel_and_SingleLabel = []
Match_7_MultiLabel_and_SingleLabel_Baseline = []
Match_7_MultiLabel_and_SingleLabel_Upperbound = []

Match_6_MultiLabel = []
Match_6_MultiLabel_Baseline = []
Match_6_MultiLabel_Upperbound = []
Match_6_SingleLabel = []
Match_6_SingleLabel_Baseline = []
Match_6_SingleLabel_Upperbound = []
Match_6_MultiLabel_and_SingleLabel = []
Match_6_MultiLabel_and_SingleLabel_Baseline = []
Match_6_MultiLabel_and_SingleLabel_Upperbound = []

print(len(test_multi_label_out))
print(len(test_single_label_out))

Categorical_Accuracy_Mix = []
Categorical_Accuracy_MultiLabel = []
Categorical_Accuracy_SingleLabel = []

8
8


In [64]:
for i in range(0, 10):
    
    Save_Model_Dir = Dir + "DCASE/saved_models/"
    
    model = build_model()
    model = train_model(model)
    timestamp = calendar.timegm(time.gmtime())
    save_model(model, timestamp, Save_Model_Dir)
    model_load = load_model(timestamp, Save_Model_Dir)
    
    print("i = ", i) 
    MAP8_multi, B_MAP8_multi, U_MAP8_multi = Compute_Mean_Average_Precision(8, model_load, X_test_MultiLabel, Y_test_MultiLabel)
    MAP8_single, B_MAP8_single, U_MAP8_single = Compute_Mean_Average_Precision(8, model_load, X_test_SingleLabel, Y_test_SingleLabel)
    MAP8, B_MAP8, U_MAP8 = Compute_Mean_Average_Precision(8, model_load, X_test, Y_test)
    print("8")
    print(MAP8_multi)
    print(B_MAP8_multi)
    print(U_MAP8_multi)
    print(MAP8_single)
    print(B_MAP8_single)
    print(U_MAP8_single)
    print(MAP8)
    print(B_MAP8)
    print(U_MAP8)
    Match_8_MultiLabel.append(MAP8_multi)
    Match_8_MultiLabel_Baseline.append(B_MAP8_multi)
    Match_8_MultiLabel_Upperbound.append(U_MAP8_multi)
    Match_8_SingleLabel.append(MAP8_single)
    Match_8_SingleLabel_Baseline.append(B_MAP8_single)
    Match_8_SingleLabel_Upperbound.append(U_MAP8_single)
    Match_8_MultiLabel_and_SingleLabel.append(MAP8)
    Match_8_MultiLabel_and_SingleLabel_Baseline.append(B_MAP8)
    Match_8_MultiLabel_and_SingleLabel_Upperbound.append(U_MAP8)
    

    MAP7_multi, B_MAP7_multi, U_MAP7_multi = Compute_Mean_Average_Precision(7, model_load, X_test_MultiLabel, Y_test_MultiLabel)
    MAP7_single, B_MAP7_single, U_MAP7_single = Compute_Mean_Average_Precision(7, model_load, X_test_SingleLabel, Y_test_SingleLabel)
    MAP7, B_MAP7, U_MAP7 = Compute_Mean_Average_Precision(7, model_load, X_test, Y_test)
    print("7")
    print(MAP7_multi)
    print(B_MAP7_multi)
    print(U_MAP7_multi)
    print(MAP7_single)
    print(B_MAP7_single)
    print(U_MAP7_single)
    print(MAP7)
    print(B_MAP7)
    print(U_MAP7)
    Match_7_MultiLabel.append(MAP7_multi)
    Match_7_MultiLabel_Baseline.append(B_MAP7_multi)
    Match_7_MultiLabel_Upperbound.append(U_MAP7_multi)
    Match_7_SingleLabel.append(MAP7_single)
    Match_7_SingleLabel_Baseline.append(B_MAP7_single)
    Match_7_SingleLabel_Upperbound.append(U_MAP7_single)
    Match_7_MultiLabel_and_SingleLabel.append(MAP7)
    Match_7_MultiLabel_and_SingleLabel_Baseline.append(B_MAP7)
    Match_7_MultiLabel_and_SingleLabel_Upperbound.append(U_MAP7)

    
    MAP6_multi, B_MAP6_multi, U_MAP6_multi = Compute_Mean_Average_Precision(6, model_load, X_test_MultiLabel, Y_test_MultiLabel)
    MAP6_single, B_MAP6_single, U_MAP6_single = Compute_Mean_Average_Precision(6, model_load, X_test_SingleLabel, Y_test_SingleLabel)
    MAP6, B_MAP6, U_MAP6 = Compute_Mean_Average_Precision(6, model_load, X_test, Y_test)
    print("6")
    print(MAP6_multi)
    print(B_MAP6_multi)
    print(U_MAP6_multi)
    print(MAP6_single)
    print(B_MAP6_single)
    print(U_MAP6_single)
    print(MAP6)
    print(B_MAP6)
    print(U_MAP6)
    Match_6_MultiLabel.append(MAP6_multi)
    Match_6_MultiLabel_Baseline.append(B_MAP6_multi)
    Match_6_MultiLabel_Upperbound.append(U_MAP6_multi)
    Match_6_SingleLabel.append(MAP6_single)
    Match_6_SingleLabel_Baseline.append(B_MAP6_single)
    Match_6_SingleLabel_Upperbound.append(U_MAP6_single)
    Match_6_MultiLabel_and_SingleLabel.append(MAP6)
    Match_6_MultiLabel_and_SingleLabel_Baseline.append(B_MAP6)
    Match_6_MultiLabel_and_SingleLabel_Upperbound.append(U_MAP6)
    
#     Categorical_Accuracy_Evaluation(model, te_pairs_multi_label, test_multi_label_out)
#     Categorical_Accuracy_Evaluation(model, te_pairs, test_out)

    
    Categorical_Accuracy_Mix.append(Categorical_Accuracy_Evaluation(model, te_pairs, test_out))
    Categorical_Accuracy_MultiLabel.append(Categorical_Accuracy_Evaluation(model, te_pairs_multi_label, test_multi_label_out))
    Categorical_Accuracy_SingleLabel.append(Categorical_Accuracy_Evaluation(model, te_pairs_single_label, test_single_label_out))
    
    


Epoch 1/3
246071/246071 [==============================] - 70s 286us/step - loss: 2.5535 - lambda_9_loss: 0.6213 - lambda_10_loss: 0.4400 - lambda_11_loss: 0.1929 - lambda_12_loss: 0.1466 - lambda_13_loss: 0.5158 - lambda_14_loss: 0.1501 - lambda_15_loss: 0.3748 - lambda_16_loss: 0.1119 - lambda_9_categorical_accuracy: 0.7193 - lambda_9_f1_m: 0.6234 - lambda_9_precision_m: 0.7851 - lambda_9_recall_m: 0.5214 - lambda_10_categorical_accuracy: 0.8126 - lambda_10_f1_m: 0.7360 - lambda_10_precision_m: 0.8757 - lambda_10_recall_m: 0.6376 - lambda_11_categorical_accuracy: 0.9273 - lambda_11_f1_m: 0.9118 - lambda_11_precision_m: 0.9483 - lambda_11_recall_m: 0.8795 - lambda_12_categorical_accuracy: 0.9492 - lambda_12_f1_m: 0.9382 - lambda_12_precision_m: 0.9631 - lambda_12_recall_m: 0.9150 - lambda_13_categorical_accuracy: 0.7850 - lambda_13_f1_m: 0.7124 - lambda_13_precision_m: 0.8406 - lambda_13_recall_m: 0.6212 - lambda_14_categorical_accuracy: 0.9505 - lambda_14_f1_m: 0.9311 - lambda_14_pr

246071/246071 [==============================] - 72s 291us/step - loss: 2.5678 - lambda_17_loss: 0.6224 - lambda_18_loss: 0.4340 - lambda_19_loss: 0.1925 - lambda_20_loss: 0.1475 - lambda_21_loss: 0.5131 - lambda_22_loss: 0.1486 - lambda_23_loss: 0.3847 - lambda_24_loss: 0.1250 - lambda_17_categorical_accuracy: 0.7181 - lambda_17_f1_m: 0.6056 - lambda_17_precision_m: 0.7887 - lambda_17_recall_m: 0.4944 - lambda_18_categorical_accuracy: 0.8160 - lambda_18_f1_m: 0.7369 - lambda_18_precision_m: 0.8782 - lambda_18_recall_m: 0.6377 - lambda_19_categorical_accuracy: 0.9283 - lambda_19_f1_m: 0.9249 - lambda_19_precision_m: 0.9266 - lambda_19_recall_m: 0.9239 - lambda_20_categorical_accuracy: 0.9487 - lambda_20_f1_m: 0.9355 - lambda_20_precision_m: 0.9637 - lambda_20_recall_m: 0.9094 - lambda_21_categorical_accuracy: 0.7883 - lambda_21_f1_m: 0.6992 - lambda_21_precision_m: 0.8476 - lambda_21_recall_m: 0.5978 - lambda_22_categorical_accuracy: 0.9510 - lambda_22_f1_m: 0.9229 - lambda_22_precisio

246071/246071 [==============================] - 69s 282us/step - loss: 1.7464 - lambda_25_loss: 0.4991 - lambda_26_loss: 0.2658 - lambda_27_loss: 0.1075 - lambda_28_loss: 0.0712 - lambda_29_loss: 0.3685 - lambda_30_loss: 0.0874 - lambda_31_loss: 0.2900 - lambda_32_loss: 0.0569 - lambda_25_categorical_accuracy: 0.7861 - lambda_25_f1_m: 0.5859 - lambda_25_precision_m: 0.8808 - lambda_25_recall_m: 0.4403 - lambda_26_categorical_accuracy: 0.8956 - lambda_26_f1_m: 0.7570 - lambda_26_precision_m: 0.9590 - lambda_26_recall_m: 0.6264 - lambda_27_categorical_accuracy: 0.9569 - lambda_27_f1_m: 0.8912 - lambda_27_precision_m: 0.9909 - lambda_27_recall_m: 0.8104 - lambda_28_categorical_accuracy: 0.9744 - lambda_28_f1_m: 0.9502 - lambda_28_precision_m: 0.9872 - lambda_28_recall_m: 0.9161 - lambda_29_categorical_accuracy: 0.8561 - lambda_29_f1_m: 0.7349 - lambda_29_precision_m: 0.9190 - lambda_29_recall_m: 0.6139 - lambda_30_categorical_accuracy: 0.9677 - lambda_30_f1_m: 0.9045 - lambda_30_precisio

246071/246071 [==============================] - 69s 281us/step - loss: 1.4153 - lambda_33_loss: 0.4011 - lambda_34_loss: 0.2043 - lambda_35_loss: 0.0797 - lambda_36_loss: 0.0508 - lambda_37_loss: 0.3013 - lambda_38_loss: 0.0717 - lambda_39_loss: 0.2525 - lambda_40_loss: 0.0540 - lambda_33_categorical_accuracy: 0.8371 - lambda_33_f1_m: 0.5823 - lambda_33_precision_m: 0.9349 - lambda_33_recall_m: 0.4243 - lambda_34_categorical_accuracy: 0.9227 - lambda_34_f1_m: 0.7462 - lambda_34_precision_m: 0.9801 - lambda_34_recall_m: 0.6035 - lambda_35_categorical_accuracy: 0.9696 - lambda_35_f1_m: 0.8837 - lambda_35_precision_m: 0.9970 - lambda_35_recall_m: 0.7942 - lambda_36_categorical_accuracy: 0.9816 - lambda_36_f1_m: 0.9125 - lambda_36_precision_m: 0.9979 - lambda_36_recall_m: 0.8412 - lambda_37_categorical_accuracy: 0.8873 - lambda_37_f1_m: 0.7481 - lambda_37_precision_m: 0.9470 - lambda_37_recall_m: 0.6194 - lambda_38_categorical_accuracy: 0.9734 - lambda_38_f1_m: 0.9454 - lambda_38_precisio

246071/246071 [==============================] - 70s 283us/step - loss: 1.4129 - lambda_41_loss: 0.4085 - lambda_42_loss: 0.2087 - lambda_43_loss: 0.0826 - lambda_44_loss: 0.0528 - lambda_45_loss: 0.2977 - lambda_46_loss: 0.0723 - lambda_47_loss: 0.2439 - lambda_48_loss: 0.0463 - lambda_41_categorical_accuracy: 0.8318 - lambda_41_f1_m: 0.5713 - lambda_41_precision_m: 0.9338 - lambda_41_recall_m: 0.4130 - lambda_42_categorical_accuracy: 0.9205 - lambda_42_f1_m: 0.7496 - lambda_42_precision_m: 0.9776 - lambda_42_recall_m: 0.6087 - lambda_43_categorical_accuracy: 0.9683 - lambda_43_f1_m: 0.8942 - lambda_43_precision_m: 0.9950 - lambda_43_recall_m: 0.8126 - lambda_44_categorical_accuracy: 0.9808 - lambda_44_f1_m: 0.9235 - lambda_44_precision_m: 0.9973 - lambda_44_recall_m: 0.8603 - lambda_45_categorical_accuracy: 0.8913 - lambda_45_f1_m: 0.7506 - lambda_45_precision_m: 0.9483 - lambda_45_recall_m: 0.6223 - lambda_46_categorical_accuracy: 0.9733 - lambda_46_f1_m: 0.9079 - lambda_46_precisio

8
[0.0761904761904762, 0.10273809523809524, 0.11881188586545725, 0.11240955028579822, 0.10713177936051102, 0.0974883251894102]
[0.0, 0.03801587301587302, 0.05404383975812547, 0.04214168630937303, 0.06447079537371635, 0.04245773377873932]
[0.9428571428571428, 0.9428571428571428, 0.9428571428571428, 0.9428571428571428, 0.9428571428571428, 0.9428571428571428]
[0.8070739549839229, 0.8226598785280456, 0.7955259698556112, 0.7785633954319758, 0.7758184732102795, 0.7740114528938935]
[0.18971061093247588, 0.33080564487316894, 0.3380757705011262, 0.26683069921206093, 0.25473453442474503, 0.2244639554948901]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.6225961538461539, 0.6409488514957263, 0.6247207323100195, 0.6104236027869069, 0.6070393798106984, 0.6032544134492522]
[0.18028846153846154, 0.27028912927350435, 0.2649040528337402, 0.22670798181150628, 0.21423016316465887, 0.1863984259060151]
[0.9855769230769231, 0.9855769230769231, 0.9855769230769231, 0.9855769230769231, 0.9855769230769231, 0.985576923076923

6
[0.8857142857142857, 0.9058862433862436, 0.9038720095622024, 0.9009442312616194, 0.8963492760840128, 0.8885273470765674]
[0.5523809523809524, 0.5913095238095238, 0.5963684252360558, 0.5455110793464485, 0.5152880859220819, 0.4979579926793596]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.9967845659163987, 0.9986602357984996, 0.9961957785376863, 0.9954813832850605, 0.9953180878622965, 0.9951157502238701]
[0.887459807073955, 0.9207752768846016, 0.9235175782531657, 0.9080171519688056, 0.9006883906528583, 0.898341786128782]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.96875, 0.9752437232905982, 0.9728929041568547, 0.9716198425099135, 0.9703379791201817, 0.9682124273140944]
[0.7788461538461539, 0.8592648237179481, 0.8378962844708752, 0.8048530331349981, 0.8038431359661711, 0.7970649091894904]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.6490192, 0.8382837, 0.95367223, 0.9380607, 0.84335244, 0.89908254, 0.8486239, 0.96459526]
[0.30654076, 0.59511745, 0.8394749, 0.847766, 0.39359742, 0.77015203, 0.36964533, 0.88899124]
[0.42

246071/246071 [==============================] - 84s 342us/step - loss: 2.5440 - lambda_73_loss: 0.6199 - lambda_74_loss: 0.4266 - lambda_75_loss: 0.1920 - lambda_76_loss: 0.1368 - lambda_77_loss: 0.5149 - lambda_78_loss: 0.1505 - lambda_79_loss: 0.3827 - lambda_80_loss: 0.1206 - lambda_73_categorical_accuracy: 0.7183 - lambda_73_f1_m: 0.6119 - lambda_73_precision_m: 0.7848 - lambda_73_recall_m: 0.5077 - lambda_74_categorical_accuracy: 0.8193 - lambda_74_f1_m: 0.7310 - lambda_74_precision_m: 0.8849 - lambda_74_recall_m: 0.6256 - lambda_75_categorical_accuracy: 0.9303 - lambda_75_f1_m: 0.9098 - lambda_75_precision_m: 0.9529 - lambda_75_recall_m: 0.8730 - lambda_76_categorical_accuracy: 0.9514 - lambda_76_f1_m: 0.9409 - lambda_76_precision_m: 0.9655 - lambda_76_recall_m: 0.9179 - lambda_77_categorical_accuracy: 0.7871 - lambda_77_f1_m: 0.7034 - lambda_77_precision_m: 0.8464 - lambda_77_recall_m: 0.6047 - lambda_78_categorical_accuracy: 0.9493 - lambda_78_f1_m: 0.9241 - lambda_78_precisio

246071/246071 [==============================] - 90s 365us/step - loss: 2.5420 - lambda_81_loss: 0.6229 - lambda_82_loss: 0.4359 - lambda_83_loss: 0.1955 - lambda_84_loss: 0.1374 - lambda_85_loss: 0.5077 - lambda_86_loss: 0.1494 - lambda_87_loss: 0.3805 - lambda_88_loss: 0.1127 - lambda_81_categorical_accuracy: 0.7170 - lambda_81_f1_m: 0.6266 - lambda_81_precision_m: 0.7758 - lambda_81_recall_m: 0.5279 - lambda_82_categorical_accuracy: 0.8150 - lambda_82_f1_m: 0.7234 - lambda_82_precision_m: 0.8852 - lambda_82_recall_m: 0.6144 - lambda_83_categorical_accuracy: 0.9270 - lambda_83_f1_m: 0.9250 - lambda_83_precision_m: 0.9241 - lambda_83_recall_m: 0.9264 - lambda_84_categorical_accuracy: 0.9520 - lambda_84_f1_m: 0.9353 - lambda_84_precision_m: 0.9702 - lambda_84_recall_m: 0.9035 - lambda_85_categorical_accuracy: 0.7893 - lambda_85_f1_m: 0.7025 - lambda_85_precision_m: 0.8506 - lambda_85_recall_m: 0.6018 - lambda_86_categorical_accuracy: 0.9498 - lambda_86_f1_m: 0.9482 - lambda_86_precisio

In [65]:
Result_Dir = "/mnt/tmp/DCASE_20190816/Results4/"

def WritePickle():
    
    db = {} 
    db['Match_8_MultiLabel'] = Match_8_MultiLabel 
    db['Match_8_MultiLabel_Baseline'] = Match_8_MultiLabel_Baseline
    db['Match_8_MultiLabel_Upperbound'] = Match_8_MultiLabel_Upperbound
    db['Match_8_SingleLabel'] = Match_8_SingleLabel
    db['Match_8_SingleLabel_Baseline'] = Match_8_SingleLabel_Baseline
    db['Match_8_SingleLabel_Upperbound'] = Match_8_SingleLabel_Upperbound
    db['Match_8_MultiLabel_and_SingleLabel'] = Match_8_MultiLabel_and_SingleLabel
    db['Match_8_MultiLabel_and_SingleLabel_Baseline'] = Match_8_MultiLabel_and_SingleLabel_Baseline
    db['Match_8_MultiLabel_and_SingleLabel_Upperbound'] = Match_8_MultiLabel_and_SingleLabel_Upperbound
    
    db['Match_7_MultiLabel'] = Match_7_MultiLabel 
    db['Match_7_MultiLabel_Baseline'] = Match_7_MultiLabel_Baseline
    db['Match_7_MultiLabel_Upperbound'] = Match_7_MultiLabel_Upperbound
    db['Match_7_SingleLabel'] = Match_7_SingleLabel
    db['Match_7_SingleLabel_Baseline'] = Match_7_SingleLabel_Baseline
    db['Match_7_SingleLabel_Upperbound'] = Match_7_SingleLabel_Upperbound
    db['Match_7_MultiLabel_and_SingleLabel'] = Match_7_MultiLabel_and_SingleLabel
    db['Match_7_MultiLabel_and_SingleLabel_Baseline'] = Match_7_MultiLabel_and_SingleLabel_Baseline
    db['Match_7_MultiLabel_and_SingleLabel_Upperbound'] = Match_7_MultiLabel_and_SingleLabel_Upperbound
    
    db['Match_6_MultiLabel'] = Match_6_MultiLabel 
    db['Match_6_MultiLabel_Baseline'] = Match_6_MultiLabel_Baseline
    db['Match_6_MultiLabel_Upperbound'] = Match_6_MultiLabel_Upperbound
    db['Match_6_SingleLabel'] = Match_6_SingleLabel
    db['Match_6_SingleLabel_Baseline'] = Match_6_SingleLabel_Baseline
    db['Match_6_SingleLabel_Upperbound'] = Match_6_SingleLabel_Upperbound
    db['Match_6_MultiLabel_and_SingleLabel'] = Match_6_MultiLabel_and_SingleLabel
    db['Match_6_MultiLabel_and_SingleLabel_Baseline'] = Match_6_MultiLabel_and_SingleLabel_Baseline
    db['Match_6_MultiLabel_and_SingleLabel_Upperbound'] = Match_6_MultiLabel_and_SingleLabel_Upperbound
    
#     db['Categorical_Accuracy_Mix'] = Categorical_Accuracy_Mix
#     db['Categorical_Accuracy_MultiLabel'] = Categorical_Accuracy_MultiLabel 
#     db['Categorical_Accuracy_SingleLabel'] = Categorical_Accuracy_SingleLabel 
    
    # Its important to use binary mode 
    dbfile = open(Result_Dir + 'MAP_20190826_SONYC_Concat.pkl', 'ab') 
      
    # source, destination 
    pickle.dump(db, dbfile)                      
    dbfile.close() 

In [66]:
WritePickle()

In [45]:
# Top_K = 30
# MAP_1, MAP_5, MAP_10, MAP_30, MAP_50, MAP_100, MAP_database_size, B_MAP_1, B_MAP_5, B_MAP_10, B_MAP_30, B_MAP_50, B_MAP_100, B_MAP_database_size = Compute_Mean_Average_Precision()
# MAP7, B_MAP7 = Compute_Mean_Average_Precision(7)
# MAP8, B_MAP8 = Compute_Mean_Average_Precision(8)
# MAP5, B_MAP5 = Compute_Mean_Average_Precision(5)
# MAP4, B_MAP4 = Compute_Mean_Average_Precision(4)
# MAP3, B_MAP3 = Compute_Mean_Average_Precision(3)
# MAP2, B_MAP2 = Compute_Mean_Average_Precision(2)

MAP8, B_MAP8 = Compute_Mean_Average_Precision(8)
print("8")
print(MAP8)
print(B_MAP8)

MAP7, B_MAP7 = Compute_Mean_Average_Precision(7)
print("7")
print(MAP7)
print(B_MAP7)

MAP6, B_MAP6 = Compute_Mean_Average_Precision(6)
print("6")
print(MAP6)
print(B_MAP6)

# print(MAP_1, MAP_5, MAP_10, MAP_30, MAP_50, MAP_100, MAP_database_size)
# print(B_MAP_1, B_MAP_5, B_MAP_10, B_MAP_30, B_MAP_50, B_MAP_100, B_MAP_database_size)

8
[0.6057692307692307, 0.6257912660256406, 0.6047302418563588, 0.5944325613066028, 0.5889631675069973, 0.5850503000699397, 0.550015140379019]
[0.14903846153846154, 0.26382545405982905, 0.2555905162545788, 0.22269529511216138, 0.2072340668833888, 0.17595704885688898, 0.15408858695344477]
7
[0.7884615384615384, 0.8110476762820511, 0.8103971581287432, 0.805554538284894, 0.7987160901785304, 0.7942037257052313, 0.7082310857350296]
[0.31971153846153844, 0.4330061431623929, 0.3954093851750101, 0.35766702009880424, 0.3427909885692282, 0.3079493438732663, 0.2845199902525964]
6
[0.9591346153846154, 0.9716947115384614, 0.970516622892925, 0.9703122661458111, 0.9701482315641026, 0.9696372022565881, 0.9187962091667313]
[0.78125, 0.8723991720085469, 0.8320451319995588, 0.8161183931831247, 0.8071893150935683, 0.807586561805813, 0.79475775845953]


In [45]:
# te_pairs.shape
# input_1 = te_pairs[:num_test_samples, 0, :10, :128]
# input_2 = te_pairs[:num_test_samples, 1, :10, :128]
# print(input_1.shape)

(39458, 10, 128)


In [46]:
# print(model.test_on_batch([input_1, input_2], [test_out_1, test_out_2,test_out_3, test_out_4,test_out_5,
#                                                test_out_6,test_out_7,test_out_8]))

[4.209803, 1.095926, 0.6935152, 0.34361097, 0.32452086, 0.5361294, 0.4399827, 0.49747676, 0.2786411, 0.62955546, 0.48643556, 0.6767403, 0.37966952, 0.8509301, 0.8375579, 0.8858144, 0.7942876, 0.9436363, 0.9242829, 0.96433586, 0.8874246, 0.9329667, 0.9225408, 0.95102656, 0.8957119, 0.84616554, 0.7584867, 0.87702084, 0.6681788, 0.90688837, 0.8176071, 0.95534325, 0.7145826, 0.8361549, 0.6689337, 0.89672804, 0.53342795, 0.96302396, 0.9647405, 0.9755649, 0.9541538]


In [47]:
# Performance_tmp = model.test_on_batch([input_1, input_2], [test_out_1, test_out_2,test_out_3, test_out_4,test_out_5,
#                                                test_out_6,test_out_7,test_out_8])

In [48]:
# print(len(Performance_tmp))
# Performance = Performance_tmp[9:]
# print(len(Performance))
# # print(Performance)
# acc_summary = []
# f1_summary = []
# precision_summary = []
# recall_summary = []
# for i in range(0, 8):
# #     print(i * 4 + 0)
# #     print(Performance[i * 4 + 0])
#     acc_summary.append(Performance[i * 4 + 0])
#     f1_summary.append(Performance[i * 4 + 1])
#     precision_summary.append(Performance[i * 4 + 2])
#     recall_summary.append(Performance[i * 4 + 3])

41
32


In [49]:
# acc_summary       ## acc_summary

[0.62955546,
 0.8509301,
 0.9436363,
 0.9329667,
 0.84616554,
 0.90688837,
 0.8361549,
 0.96302396]

In [50]:
# precision_summary   #0.89725400125

[0.6767403,
 0.8858144,
 0.96433586,
 0.95102656,
 0.87702084,
 0.95534325,
 0.89672804,
 0.9755649]

In [51]:
# recall_summary     ## 0.7936096625

[0.37966952,
 0.7942876,
 0.8874246,
 0.8957119,
 0.6681788,
 0.7145826,
 0.53342795,
 0.9541538]

In [52]:
# f1_summary

[0.48643556,
 0.8375579,
 0.9242829,
 0.9225408,
 0.7584867,
 0.8176071,
 0.6689337,
 0.9647405]

In [49]:
# test = model.predict_on_batch([input_1, input_2])
# test_out_1.shape

In [50]:
# print(test[0][0], " " , test_out_1[0])
# print(test[0][1], " " , test_out_2[0])
# print(test[0][2], " " , test_out_3[0])
# print(test[0][3], " " , test_out_4[0])
# print(test[0][4], " " , test_out_5[0])
# print(test[0][5], " " , test_out_6[0])
# print(test[0][6], " " , test_out_7[0])
# print(test[0][7], " " , test_out_8[0])
# print(test[0][8], " " , test_out_9[0])
# print(test[0][9], " " , test_out_10[0])

In [59]:
# for i in range(0, 10):
#     print(test[i][0], " " , test_out_1[i])
#     print(test[i][1], " " , test_out_2[i])
#     print(test[i][2], " " , test_out_3[i])
#     print(test[i][3], " " , test_out_4[i])
#     print(test[i][4], " " , test_out_5[i])
#     print(test[i][5], " " , test_out_6[i])
#     print(test[i][6], " " , test_out_7[i])
#     print(test[i][7], " " , test_out_8[i])
#     print(test[i][8], " " , test_out_9[i])
#     print(test[i][9], " " , test_out_10[i])
#     print()

In [60]:
print(test_out_1.shape)
print(test_out_2.shape)
print(test_out_3.shape)
print(test_out_4.shape)
print(test_out_5.shape)
print(test_out_6.shape)
print(test_out_7.shape)
print(test_out_8.shape)
# print(test_out_9.shape)
# print(test_out_10.shape)

(39458, 3)
(39458, 3)
(39458, 3)
(39458, 3)
(39458, 3)
(39458, 3)
(39458, 3)
(39458, 3)


In [61]:
# tmp = test_out_1
# number = num_test_samples * 3
# tmp2 = tmp.reshape(number)
# print(tmp2.shape)
# print(tmp[0], " ", tmp[1], " ", tmp[2], " ", tmp[3])
# print(tmp2[0], " ", tmp2[1], " ", tmp2[2], " ", tmp2[3], " ", tmp2[4], " ", tmp2[5], " ", tmp2[6], " ", tmp2[7])


# pred_out_1 = np.zeros((num_samples, 3))
# for i in range(num_samples):
#     pred_out_1[i][0] = 1
# pred_out_1 = pred_out_1.reshape(number)        

# tmp3 = K.variable(tmp2)
# pred_out_2 = K.variable(pred_out_1)

# score = f1_m(tmp3, pred_out_2)
# print(score)
# print(K.eval(score))

In [62]:
import random
# random.uniform(0, 1)

num_samples = num_test_samples
number = num_test_samples * 3

# pred_out_1 = np.random.rand(num_samples, 3)
# binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
# categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
# print(K.eval(binary_acc))
# print(K.eval(categorical_acc))
# print()

# pred_out_1 = np.zeros((num_samples, 3))
# # pred_out_1 = np.ones((100421, 3))
# binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
# categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
# print(K.eval(binary_acc))
# print(K.eval(categorical_acc))
# print()

# pred_out_1 = np.ones((num_samples, 3))
# binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
# categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
# print(K.eval(binary_acc))
# print(K.eval(categorical_acc))
# print()

# pred_out_1 = np.zeros((num_samples, 3))
# for i in range(num_samples):
#     pred_out_1[i][0] = 1
# # pred_out_1 = np.ones((100421, 3))
# binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
# categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
# print(K.eval(binary_acc))
# print(K.eval(categorical_acc))
# print()

# pred_out_1 = np.zeros((num_samples, 3))
# for i in range(num_samples):
#     pred_out_1[i][1] = 1
# # pred_out_1 = np.ones((100421, 3))
# binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
# categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
# print(K.eval(binary_acc))
# print(K.eval(categorical_acc))
# print()

# pred_out_1 = np.zeros((num_samples, 3))
# for i in range(num_samples):
#     pred_out_1[i][2] = 1
# # pred_out_1 = np.ones((100421, 3))
# binary_acc = K.mean(K.equal(test_out_1, K.round(pred_out_1)))
# categorical_acc = K.mean(K.equal(K.argmax(test_out_1, axis=-1), K.argmax(pred_out_1, axis=-1)))
# print(K.eval(binary_acc))
# print(K.eval(categorical_acc))
# print()

In [63]:
def get_Precision_Recall_F1(test_out, pred_out):
    number = num_test_samples * 3
#     test_out_one_dimension = test_out.reshape(number)
#     pred_out_one_dimension = pred_out.reshape(number)

    test_out_one_dimension = test_out
    pred_out_one_dimension = pred_out

    true_label = K.variable(test_out_one_dimension)
    pred_label = K.variable(pred_out_one_dimension)

    precision = precision_m(true_label, pred_label)
    recall = recall_m(true_label, pred_label)
    f_score = f1_m(true_label, pred_label)
#     print(score)
#     print(K.eval(score))
    return precision, recall, f_score
    

def FirstCase(test_out):
    pred_out = np.zeros((num_samples, 3))
    for i in range(num_samples):
        pred_out[i][0] = 1
    # pred_out_1 = np.ones((100421, 3))
    binary_acc = K.mean(K.equal(test_out, K.round(pred_out)))
    categorical_acc = K.mean(K.equal(K.argmax(test_out, axis=-1), K.argmax(pred_out, axis=-1)))
    precision, recall, f_score = get_Precision_Recall_F1(test_out, pred_out)
#     print(K.eval(binary_acc))
#     print(K.eval(categorical_acc))
    return categorical_acc, precision, recall, f_score

def SecondCase(test_out):
    pred_out = np.zeros((num_samples, 3))
    for i in range(num_samples):
        pred_out[i][1] = 1
    # pred_out_1 = np.ones((100421, 3))
    binary_acc = K.mean(K.equal(test_out, K.round(pred_out)))
    categorical_acc = K.mean(K.equal(K.argmax(test_out, axis=-1), K.argmax(pred_out, axis=-1)))
    precision, recall, f_score = get_Precision_Recall_F1(test_out, pred_out)
#     print(K.eval(binary_acc))
#     print(K.eval(categorical_acc))
    return categorical_acc, precision, recall, f_score

def ThirdCase(test_out):
    pred_out = np.zeros((num_samples, 3))
    for i in range(num_samples):
        pred_out[i][2] = 1
    # pred_out_1 = np.ones((100421, 3))
    binary_acc = K.mean(K.equal(test_out, K.round(pred_out)))
    categorical_acc = K.mean(K.equal(K.argmax(test_out, axis=-1), K.argmax(pred_out, axis=-1)))
    precision, recall, f_score = get_Precision_Recall_F1(test_out, pred_out)
#     print(K.eval(binary_acc))
#     print(K.eval(categorical_acc))
    return categorical_acc, precision, recall, f_score


print('1111111')
baseline1 = []
baseline1_precision = []
baseline1_recall = []
baseline1_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_1)
# print(K.eval(categorical_acc),"  ", K.eval(precision),"  ", K.eval(recall),"  ", K.eval(f1))
baseline1.append(K.eval(categorical_acc))
baseline1_precision.append(K.eval(precision))
baseline1_recall.append(K.eval(recall))
# baseline1_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_1)
baseline1.append(K.eval(categorical_acc))
baseline1_precision.append(K.eval(precision))
baseline1_recall.append(K.eval(recall))
# baseline1_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_1)
baseline1.append(K.eval(categorical_acc))
baseline1_precision.append(K.eval(precision))
baseline1_recall.append(K.eval(recall))
# baseline1_f1.append(K.eval(f1))


print('2222222')
baseline2 = []
baseline2_precision = []
baseline2_recall = []
baseline2_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_2)
baseline2.append(K.eval(categorical_acc))
baseline2_precision.append(K.eval(precision))
baseline2_recall.append(K.eval(recall))
# baseline2_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_2)
baseline2.append(K.eval(categorical_acc))
baseline2_precision.append(K.eval(precision))
baseline2_recall.append(K.eval(recall))
# baseline2_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_2)
baseline2.append(K.eval(categorical_acc))
baseline2_precision.append(K.eval(precision))
baseline2_recall.append(K.eval(recall))
# baseline2_f1.append(K.eval(f1))
print()



print('3333333')
baseline3 = []
baseline3_precision = []
baseline3_recall = []
baseline3_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_3)
baseline3.append(K.eval(categorical_acc))
baseline3_precision.append(K.eval(precision))
baseline3_recall.append(K.eval(recall))
# baseline3_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_3)
baseline3.append(K.eval(categorical_acc))
baseline3_precision.append(K.eval(precision))
baseline3_recall.append(K.eval(recall))
# baseline3_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_3)
baseline3.append(K.eval(categorical_acc))
baseline3_precision.append(K.eval(precision))
baseline3_recall.append(K.eval(recall))
# baseline3_f1.append(K.eval(f1))
print()


print('4444444')
baseline4 = []
baseline4_precision = []
baseline4_recall = []
baseline4_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_4)
baseline4.append(K.eval(categorical_acc))
baseline4_precision.append(K.eval(precision))
baseline4_recall.append(K.eval(recall))
# baseline4_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_4)
baseline4.append(K.eval(categorical_acc))
baseline4_precision.append(K.eval(precision))
baseline4_recall.append(K.eval(recall))
# baseline4_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_4)
baseline4.append(K.eval(categorical_acc))
baseline4_precision.append(K.eval(precision))
baseline4_recall.append(K.eval(recall))
# baseline4_f1.append(K.eval(f1))
print()


print('5555555')
baseline5 = []
baseline5_precision = []
baseline5_recall = []
baseline5_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_5)
baseline5.append(K.eval(categorical_acc))
baseline5_precision.append(K.eval(precision))
baseline5_recall.append(K.eval(recall))
# baseline5_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_5)
baseline5.append(K.eval(categorical_acc))
baseline5_precision.append(K.eval(precision))
baseline5_recall.append(K.eval(recall))
# baseline5_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_5)
baseline5.append(K.eval(categorical_acc))
baseline5_precision.append(K.eval(precision))
baseline5_recall.append(K.eval(recall))
# baseline5_f1.append(K.eval(f1))
print()


print('6666666')
baseline6 = []
baseline6_precision = []
baseline6_recall = []
baseline6_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_6)
# print(categorical_acc,"  ", precision,"  ", recall,"  ", f1)
baseline6.append(K.eval(categorical_acc))
baseline6_precision.append(K.eval(precision))
baseline6_recall.append(K.eval(recall))
# baseline6_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_6)
baseline6.append(K.eval(categorical_acc))
baseline6_precision.append(K.eval(precision))
baseline6_recall.append(K.eval(recall))
# baseline6_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_6)
baseline6.append(K.eval(categorical_acc))
baseline6_precision.append(K.eval(precision))
baseline6_recall.append(K.eval(recall))
# baseline6_f1.append(K.eval(f1))
print()

print('7777777')
baseline7 = []
baseline7_precision = []
baseline7_recall = []
baseline7_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_7)
baseline7.append(K.eval(categorical_acc))
baseline7_precision.append(K.eval(precision))
baseline7_recall.append(K.eval(recall))
# baseline7_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_7)
baseline7.append(K.eval(categorical_acc))
baseline7_precision.append(K.eval(precision))
baseline7_recall.append(K.eval(recall))
# baseline7_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_7)
baseline7.append(K.eval(categorical_acc))
baseline7_precision.append(K.eval(precision))
baseline7_recall.append(K.eval(recall))
# baseline7_f1.append(K.eval(f1))
print()

print('8888888')
baseline8 = []
baseline8_precision = []
baseline8_recall = []
baseline8_f1 = []

categorical_acc, precision, recall, f1 = FirstCase(test_out_8)
baseline8.append(K.eval(categorical_acc))
baseline8_precision.append(K.eval(precision))
baseline8_recall.append(K.eval(recall))
# baseline8_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = SecondCase(test_out_8)
baseline8.append(K.eval(categorical_acc))
baseline8_precision.append(K.eval(precision))
baseline8_recall.append(K.eval(recall))
# baseline8_f1.append(K.eval(f1))

categorical_acc, precision, recall, f1 = ThirdCase(test_out_8)
baseline8.append(K.eval(categorical_acc))
baseline8_precision.append(K.eval(precision))
baseline8_recall.append(K.eval(recall))
# baseline8_f1.append(K.eval(f1))
print()

1111111
2222222

3333333

4444444

5555555

6666666

7777777

8888888



In [64]:
print(max(baseline1))
print(max(baseline2))
print(max(baseline3))
print(max(baseline4))
print(max(baseline5))
print(max(baseline6))
print(max(baseline7))
print(max(baseline8))
print()

print(max(baseline1_precision))
print(max(baseline2_precision))
print(max(baseline3_precision))
print(max(baseline4_precision))
print(max(baseline5_precision))
print(max(baseline6_precision))
print(max(baseline7_precision))
print(max(baseline8_precision))
print()

print(max(baseline1_recall))
print(max(baseline2_recall))
print(max(baseline3_recall))
print(max(baseline4_recall))
print(max(baseline5_recall))
print(max(baseline6_recall))
print(max(baseline7_recall))
print(max(baseline8_recall))
print()

# print(max(baseline1_f1))
# print(max(baseline2_f1))
# print(max(baseline3_f1))
# print(max(baseline4_f1))
# print(max(baseline5_f1))
# print(max(baseline6_f1))
# print(max(baseline7_f1))
# print(max(baseline8_f1))

0.53575957
0.8321253
0.95395106
0.91879976
0.5850271
0.89125144
0.54516196
0.9716914

0.53575957
0.8321253
0.95395106
0.91879976
0.5850271
0.89125144
0.54516196
0.9716914

0.53575957
0.8321253
0.95395106
0.91879976
0.5850271
0.89125144
0.54516196
0.9716914



In [65]:
baseline1_precision

[0.18754119, 0.27669927, 0.53575957]

In [66]:
baseline1

[0.18754119, 0.27669927, 0.53575957]

In [85]:
layer_name = 'model_3'
# layer_name = 'lambda_2'
layer_output = model.get_layer(layer_name).get_output_at(-1)
intermediate_layer_model = Model(inputs = model.input, outputs = layer_output)
embedding_left_sample = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
print(embedding_left_sample.shape)
print(embedding_left_sample[0].shape)
# get_3rd_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[3].output])
# layer_output = get_3rd_layer_output([x])[0]

ValueError: No such layer: model_3

In [49]:
embedding_left_sample[100,:,1]

array([0.6851285 , 0.6851285 , 0.6851285 , 0.6851285 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.23555274, 0.47561276, 0.44653296, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.6851285 ,
       0.6851285 , 0.6851285 , 0.6851285 ], dtype=float32)

In [50]:
embedding_left_sample[100,:,2]

array([0.45096046, 0.45096046, 0.45096046, 0.45096046, 0.37536597,
       1.7030022 , 1.5826298 , 0.4629561 , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.2483457 , 1.9628031 , 2.1863127 , 0.45096046,
       0.45096046, 0.45096046, 0.45096046], dtype=float32)

In [51]:
embedding_left_sample[101,:,1]

array([0.6851285, 0.6851285, 0.6851285, 0.6851285, 0.6851285, 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.6851285, 0.6851285, 0.6851285], dtype=float32)

In [22]:
layer_name = 'model_1'
# layer_name = 'lambda_2'
layer_output = model.get_layer(layer_name).get_output_at(-2)
intermediate_layer_model = Model(inputs = model.input, outputs = layer_output)
embedding_right_sample = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
print(embedding_right_sample.shape)
print(embedding_right_sample[0].shape)

(17820, 28, 128)
(28, 128)


In [24]:
# layer_name = 'sequential_2'
layer_name = 'lambda_1'
intermediate_layer_model = Model(inputs = model.input, outputs = model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
print(intermediate_output.shape)
print(intermediate_output[0].shape)
# get_3rd_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[3].output])
# layer_output = get_3rd_layer_output([x])[0]

(17820, 28, 128)
(28, 128)


In [1]:
# layer_name = 'sequential_2'
layer_name = 'dense_19'
intermediate_layer_model = Model(inputs = model.input, outputs = model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
print(intermediate_output.shape)
print(intermediate_output[0].shape)
print(intermediate_output)
# get_3rd_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[3].output])
# layer_output = get_3rd_layer_output([x])[0]

NameError: name 'Model' is not defined

In [54]:
# layer_name = 'sequential_2'
layer_name = 'dense_17'
intermediate_layer_model = Model(inputs = model.input, outputs = model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
print(intermediate_output.shape)intermediate_output
print(intermediate_output[0].shape)
print(intermediate_output)
# get_3rd_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[3].output])
# layer_output = get_3rd_layer_output([x])[0]

(17820, 28, 256)
(28, 256)
[[[-3.0925515e-04 -1.9468131e-04 -2.0819133e-04 ...  6.8165071e-04
   -4.0597166e-04  5.3202099e-04]
  [-3.0925515e-04 -1.9468131e-04 -2.0819133e-04 ...  6.8165071e-04
   -4.0597166e-04  5.3202099e-04]
  [-3.0925515e-04 -1.9468131e-04 -2.0819133e-04 ...  6.8165071e-04
   -4.0597166e-04  5.3202099e-04]
  ...
  [-3.0925515e-04 -1.9468131e-04 -2.0819133e-04 ...  6.8165071e-04
   -4.0597166e-04  5.3202099e-04]
  [-3.0925515e-04 -1.9468131e-04 -2.0819133e-04 ...  6.8165071e-04
   -4.0597166e-04  5.3202099e-04]
  [-3.0925515e-04 -1.9468131e-04 -2.0819133e-04 ...  6.8165071e-04
   -4.0597166e-04  5.3202099e-04]]

 [[-3.0925515e-04 -1.9468131e-04 -2.0819133e-04 ...  6.8165071e-04
   -4.0597166e-04  5.3202099e-04]
  [-3.0925515e-04 -1.9468131e-04 -2.0819133e-04 ...  6.8165071e-04
   -4.0597166e-04  5.3202099e-04]
  [-3.0925515e-04 -1.9468131e-04 -2.0819133e-04 ...  6.8165071e-04
   -4.0597166e-04  5.3202099e-04]
  ...
  [-3.0925515e-04 -1.9468131e-04 -2.0819133e-04 ..

In [59]:
print(intermediate_output[0,:,1].shape)
print(intermediate_output[0,:,2].shape)
print(intermediate_output[0,:,1])
print(intermediate_output[0,:,2])

(28,)
(28,)
[-1.94681314e-04 -1.94681314e-04 -1.94681314e-04 -1.94681314e-04
 -1.34549707e-01 -3.94289255e-01 -7.17345849e-02 -2.45965147e+00
 -3.21188593e+00 -3.73913598e+00 -3.45290446e+00 -1.40662909e+00
  6.72780126e-02  1.41041711e-01 -1.36213690e-01  1.08668245e-01
 -5.36943913e-01 -1.41414583e+00 -2.14969277e+00 -3.98283267e+00
 -4.60658026e+00 -7.92432845e-01 -4.20312285e-01 -1.16219354e+00
 -1.94681314e-04 -1.94681314e-04 -1.94681314e-04 -1.94681314e-04]
[-2.0819133e-04 -2.0819133e-04 -2.0819133e-04 -2.0819133e-04
 -8.3136223e-02  9.8499531e-01  7.7467984e-01  5.2240725e+00
  6.1536975e+00  6.2424555e+00  5.7229819e+00  3.1745930e+00
  3.5107559e-01  4.1947919e-01  3.7468690e-01  3.0922467e-01
  9.3947774e-01  3.0351021e+00  4.3607130e+00  6.1474857e+00
  5.9128881e+00  6.6322470e-01  4.1060853e-01  1.9918977e+00
 -2.0819133e-04 -2.0819133e-04 -2.0819133e-04 -2.0819133e-04]


In [25]:
# layer_name = 'sequential_2'
layer_name = 'dense_7'
intermediate_layer_model = Model(inputs = model.input, outputs = model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict([te_pairs[:, 0], te_pairs[:, 1]])
print(intermediate_output.shape)
print(intermediate_output[0].shape)
print(intermediate_output)
# get_3rd_layer_output = K.function([model.layers[0].input],
#                                   [model.layers[3].output])
# layer_output = get_3rd_layer_output([x])[0]

(17820, 28, 3)
(28, 3)
[[[-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]
  ...
  [-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]]

 [[-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]
  ...
  [-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]]

 [[-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]
  ...
  [-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]]

 ...

 [[-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]
  ...
  [-4.49974   -4.178382  -3.947397 ]
  [-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 -2.3411999 -2.2518334]]

 [[-2.3721104 -2.3411999 -2.2518334]
  [-2.3721104 

In [50]:
embedding_left_sample[0]

array([ 0.01475549, -0.        ,  1.366206  ,  0.77242154, -0.        ,
       -0.        , -0.        ,  0.57459396,  0.4573839 , -0.        ,
       -0.        , -0.        ,  1.1747633 ,  1.1977898 , -0.        ,
        1.2410812 ,  2.0744271 , -0.        ,  0.9161469 ,  1.4979397 ,
       -0.        ,  1.0084206 , -0.        ,  0.56954354, -0.        ,
        0.48894957, -0.        ,  1.1002976 , -0.        ,  1.2533776 ,
        1.5193446 , -0.        , -0.        ,  0.89788914, -0.        ,
        1.196692  , -0.        ,  0.84755296, -0.        , -0.        ,
       -0.        ,  0.01745285,  0.91383594,  0.3705776 ,  0.63194287,
       -0.        , -0.        ,  1.430736  ,  0.9066394 , -0.        ,
        1.0579292 ,  2.2219324 , -0.        , -0.        ,  0.33186612,
        0.61321115,  0.6211015 ,  2.1811585 ,  0.268836  ,  1.892229  ,
       -0.        , -0.        , -0.        , -0.        ,  1.5299281 ,
       -0.        , -0.        , -0.        , -0.        ,  2.04

In [51]:
embedding_right_sample[0]

array([-0.        , -0.        ,  1.755507  ,  1.5128931 , -0.        ,
       -0.        , -0.        ,  2.1970203 ,  1.1911812 , -0.        ,
       -0.        , -0.        ,  0.67282236,  0.7613121 , -0.        ,
        0.93436325,  1.5843751 , -0.        ,  0.09186152,  1.0789999 ,
       -0.        ,  0.56660295, -0.        ,  0.5243405 , -0.        ,
        0.89432603, -0.        ,  0.23707968, -0.        ,  1.6642395 ,
        1.6713197 , -0.        , -0.        ,  0.17635201,  1.1966051 ,
        1.3265302 , -0.        ,  1.3134676 , -0.        , -0.        ,
       -0.        ,  0.61383754,  1.600383  ,  0.77569675, -0.        ,
       -0.        , -0.        ,  1.2474933 ,  1.1334618 , -0.        ,
        1.2895768 ,  2.422282  , -0.        , -0.        , -0.        ,
        0.93306464,  0.45046002,  2.5096095 , -0.        ,  0.4466113 ,
       -0.        , -0.        , -0.        , -0.        ,  0.9290853 ,
       -0.        , -0.        , -0.        , -0.        ,  0.14

In [52]:
intermediate_output[0]

array([0.01475549, 0.        , 0.38930094, 0.74047154, 0.        ,
       0.        , 0.        , 1.6224263 , 0.7337973 , 0.        ,
       0.        , 0.        , 0.50194097, 0.43647766, 0.        ,
       0.306718  , 0.49005198, 0.        , 0.8242854 , 0.41893983,
       0.        , 0.44181764, 0.        , 0.04520303, 0.        ,
       0.40537646, 0.        , 0.8632179 , 0.        , 0.41086197,
       0.15197515, 0.        , 0.        , 0.7215371 , 1.1966051 ,
       0.12983823, 0.        , 0.46591467, 0.        , 0.        ,
       0.        , 0.5963847 , 0.6865471 , 0.40511915, 0.63194287,
       0.        , 0.        , 0.18324268, 0.22682244, 0.        ,
       0.23164761, 0.20034957, 0.        , 0.        , 0.33186612,
       0.31985348, 0.17064148, 0.32845092, 0.268836  , 1.4456177 ,
       0.        , 0.        , 0.        , 0.        , 0.6008428 ,
       0.        , 0.        , 0.        , 0.        , 1.9041868 ,
       0.33550566, 0.        , 0.72649395, 0.        , 0.60293

In [22]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    
    pred = y_pred.ravel() > 0.5     ## if I use sigmoid and cross entrpy, then I won't need to make the conversion. It is 
                                    ## always the correct order 
    
#     for i in range(0, len(pred)):
#         print(str(y_true[i]) + '  ' + str(y_pred[i]) + "      " +str(y_true[i]) + '  ' + str(pred[i]))
    return np.mean(pred == y_true)

In [24]:
# compute final accuracy on training and test sets
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred[:, 0])
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred[:, 0])

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

# print(y_pred)
# print(te_y)
# for i in range(0, len(y_pred)):
#     print(str(y_pred[i]) + '  ' + str(te_y[i]))

* Accuracy on training set: 99.82%
* Accuracy on test set: 99.11%


In [30]:
for i in range (0, len(y_pred)):
    print(str(y_pred[i][0]) + '  ' + str(tr_y[i]))

0.9999995  1
8.6426735e-07  0
0.9999998  1
2.9802322e-08  0
1.0  1
0.0  0
0.9999999  1
0.0  0
0.99999774  1
1.2516975e-06  0
0.99999976  1
2.220273e-05  0
0.9999999  1
0.0  0
0.9999998  1
0.0  0
0.9999998  1
0.0  0
0.9999994  1
0.0  0
0.99998116  1
1.1026859e-06  0
0.9999919  1
2.4795532e-05  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
0.99998665  1
0.0  0
0.9999994  1
2.0861626e-07  0
1.0  1
0.0  0
0.99999994  1
0.0  0
0.99999833  1
0.0  0
0.9999999  1
1.7881393e-07  0
1.0  1
0.9999277  0
0.99999666  1
1.1920929e-07  0
0.99999714  1
0.0  0
0.99999833  1
0.0  0
0.9992939  1
3.2782555e-06  0
0.9996321  1
0.0  0
0.9999989  1
2.0861626e-07  0
0.99999464  1
0.0003682375  0
0.9999975  1
5.9604645e-06  0
0.9999995  1
0.0  0
0.9999998  1
0.0  0
0.9999968  1
0.0  0
1.0  1
6.2584877e-07  0
0.99999994  1
0.0  0
0.99982405  1
0.0  0
0.9999908  1
0.04994294  0
0.9999778  1
0.0  0
0.99973696  1
0.0  0
0.993423  1
2.5629997e-06  0
0.99981517  1
0.0  0
0.99999845  1
0.0007657409  0
0.99

1.7881393e-07  0
0.9998975  1
5.9604645e-08  0
0.9999969  1
8.940697e-08  0
0.99999315  1
5.9604645e-08  0
0.9999949  1
0.0  0
0.9999999  1
0.0  0
0.9999988  1
0.0  0
0.9998156  1
5.9604645e-08  0
0.9999402  1
1.8179417e-05  0
0.99999774  1
0.0  0
0.9999987  1
0.0  0
0.9999993  1
0.0  0
0.9999999  1
0.0  0
0.99999857  1
0.0  0
0.99948955  1
0.0  0
0.9999849  1
9.030104e-06  0
0.9999356  1
3.2484531e-06  0
0.99991095  1
9.4771385e-06  0
0.99999964  1
0.0  0
0.99999976  1
0.0  0
0.99999654  1
0.0  0
0.9679381  1
1.4901161e-07  0
0.99824774  1
0.0  0
0.9999883  1
0.0  0
0.9999895  1
3.6358833e-06  0
0.9999998  1
2.9802322e-07  0
0.9999988  1
1.7881393e-07  0
0.99993837  1
4.172325e-07  0
0.99998224  1
5.6922436e-06  0
0.9999935  1
2.0861626e-07  0
0.99989116  1
1.6093254e-06  0
0.99999416  1
5.632639e-06  0
1.0  1
0.005143225  0
0.9999999  1
0.00010666251  0
0.99999845  1
0.9989811  0
0.9999999  1
0.0  0
0.9999883  1
1.1920929e-07  0
0.99999785  1
7.3313713e-06  0
1.0  1
1.1920929e-07  0


0.0  0
1.0  1
8.940697e-08  0
1.0  1
0.0039013922  0
0.9999912  1
0.0  0
0.99998605  1
1.9967556e-06  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
0.9954432  1
0.0  0
0.99995106  1
4.5508146e-05  0
0.99999976  1
0.0  0
0.9999659  1
0.0  0
1.0  1
0.0  0
0.9999994  1
4.1753054e-05  0
0.9984067  1
0.0  0
0.9999511  1
0.0017141402  0
0.9999945  1
0.0  0
0.9999692  1
1.4901161e-07  0
0.9999999  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
0.99997693  1
0.0  0
0.99539757  1
8.940697e-08  0
0.99970675  1
0.0051522255  0
0.99999714  1
0.0  0
1.0  1
0.0  0
0.9999976  1
0.0  0
1.0  1
2.7745962e-05  0
1.0  1
0.0  0
0.99999976  1
0.0  0
1.0  1
0.0  0
0.9999999  1
2.9802322e-08  0
0.99999976  1
0.0  0
0.9999995  1
1.4901161e-07  0
0.90870416  1
0.0016179085  0
0.966773  1
0.9972037  0
1.0  1
0.002221018  0
0.99999976  1
0.0  0
0.99999964  1
0.0380221  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
1.7881393e-07  0
0.9999995  1
0.0  0
0.9997332  1
2.3633242e-05  0
0.9999492  1

1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
0.999999  1
0.0  0
0.99992204  1
0.0  0
0.99996436  1
0.0  0
0.9999813  1
0.0073661506  0
0.9999999  1
1.4901161e-07  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
1.4901161e-07  0
0.99999964  1
0.0  0
0.9999994  1
0.0  0
1.0  1
0.0  0
0.9999845  1
0.0  0
0.999998  1
5.9604645e-08  0
0.99999475  1
0.0  0
0.9999953  1
3.2782555e-07  0
0.99999994  1
0.0  0
0.9999978  1
0.0  0
1.0  1
0.0  0
1.0  1
1.2814999e-06  0
0.99999857  1
7.176399e-05  0
0.9999999  1
0.0  0
0.9999988  1
0.0  0
0.9999994  1
0.0  0
0.9998697  1
0.0  0
0.9999621  1
0.00015765429  0
1.0  1
0.0  0
0.9999999  1
0.0  0
0.9999999  1
2.9802322e-08  0
0.9999998  1
0.0  0
0.9999943  1
7.748604e-07  0
0.9999989  1
0.0  0
1.0  1
0.0  0
1.0  1
1.1622906e-05  0
0.99999994  1
0.0  0
0.9999987  1
8.940697e-08  0
0.99999934  1
1.8715858e-05  0
1.0  1
0.0  0
1.0  1
0.0  0
0.99999964  1
0.0  0
0.99999976  1
2.3841858e-06  0
1.0  1
5.5730343e-06  0
0.9999921  1
0.0  0
0.9999913  1
0.0  0
1.

1.0  1
1.7285347e-06  0
1.0  1
0.0  0
1.0  1
0.0  0
0.9999962  1
0.0  0
0.999982  1
8.940697e-08  0
0.99992895  1
0.0  0
1.0  1
2.6136637e-05  0
1.0  1
0.0  0
0.9998797  1
0.0  0
0.9999311  1
1.013279e-06  0
0.9999953  1
6.2584877e-07  0
1.0  1
0.0  0
0.9999998  1
0.0  0
0.9999999  1
0.0  0
0.9995316  1
0.0  0
0.9456079  1
0.0  0
0.9995626  1
2.9802322e-08  0
1.0  1
0.0  0
0.9999999  1
0.0  0
0.9999999  1
0.0  0
0.9999999  1
0.0  0
1.0  1
0.0  0
0.99999917  1
0.0  0
0.9979136  1
0.0  0
0.9999651  1
0.0  0
0.9996344  1
0.0  0
0.9999999  1
0.0  0
1.0  1
2.0861626e-07  0
1.0  1
2.0861626e-07  0
1.0  1
0.0  0
1.0  1
0.0  0
0.99999905  1
0.0  0
0.99999976  1
0.0  0
0.99999976  1
4.142523e-06  0
0.9999989  1
0.0  0
0.99999994  1
0.0  0
0.9999994  1
0.0  0
0.99999964  1
0.0  0
1.0  1
0.0  0
0.9999999  1
0.7109047  0
1.0  1
0.0  0
1.0  1
0.0  0
0.9999994  1
0.0001437068  0
0.99999976  1
0.008091718  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
1.4901161e-07  0
1.0  1
0.0  0
1.0  1
0.33602667  0
0.9999

0.0  0
0.9999684  1
0.0  0
0.9999726  1
0.0  0
0.999139  1
0.0022347867  0
0.9999957  1
0.0  0
0.99999964  1
1.4901161e-07  0
0.9999998  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
0.9999989  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
0.9999997  1
0.0  0
0.99993414  1
0.0  0
0.9999819  1
3.6031008e-05  0
0.9999999  1
0.0  0
1.0  1
0.0  0
0.9999993  1
0.0  0
0.9999981  1
0.0  0
1.0  1
0.9997246  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
4.4703484e-07  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
0.9999999  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
0.9999999  1
1.4901161e-07  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.00059586763  0
1.0  1
0.0  0
0.99999917  1
1.1920929e-07  0
1.0  1
0.0  0
0.9999992  1
0.0  0
0.9999909  1
0.0  0
0.99999505  1
0.0  0
0.9999999  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
0.9999998  1
0.0  0
0.9999999  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0

1.0  1
0.0  0
0.9999974  1
0.0  0
0.9999826  1
0.0  0
1.0  1
0.0  0
0.99851656  1
0.0  0
0.9983649  1
0.0  0
0.9999942  1
0.0  0
1.0  1
0.0  0
0.9177849  1
0.0  0
0.048291743  1
0.0  0
0.99999255  1
0.0  0
1.0  1
2.0205975e-05  0
1.0  1
0.0  0
0.9999995  1
0.0  0
0.99999994  1
0.0  0
0.99994075  1
0.0  0
0.9994389  1
2.9325485e-05  0
0.99999917  1
6.1392784e-06  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
0.99999964  1
0.0  0
0.9093876  1
0.0  0
0.8466094  1
2.5421381e-05  0
0.99999964  1
2.682209e-07  0
1.0  1
0.0  0
0.9999999  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
0.99993646  1
5.632639e-06  0
0.1492294  1
0.0  0
0.12871838  1
0.0  0
1.0  1
0.0  0
0.99462044  1
0.0  0
0.95764637  1
0.0  0
0.99999905  1
0.0  0
0.9999958  1
0.0  0
0.99999654  1
1.1026859e-06  0
0.99999964  1
0.0  0
0.99998593  1
1.4901161e-07  0
0.99999565  1
0.0  0
0.9999998  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
0.9999901  1
1.4901161e-07  0
0.99999666  1
0.0  0
1.0  1
0.0  0
1.0  1
0.0  0
1

0.00097078085  0
0.7237919  1
5.662441e-07  0
0.6899057  1
0.0719122  0
0.9989346  1
0.0  0
0.9999877  1
0.0  0
0.9999994  1
2.9802322e-08  0
1.0  1
4.4703484e-06  0
0.99999845  1
0.0  0
0.999993  1
0.0  0
1.0  1
2.9802322e-08  0
1.0  1
0.0  0
0.9999565  1
2.9802322e-07  0
0.9966965  1
0.0  0
0.99993193  1
9.536743e-07  0
0.9788773  1
0.0  0
0.9990524  1
0.0  0
0.9999994  1
0.0  0
1.0  1
0.0  0
1.0  1
2.0861626e-06  0
1.0  1
2.9802322e-08  0
1.0  1
0.0  0
0.99999523  1
0.0  0
0.9999999  1
0.039996  0
1.0  1
0.0  0
1.0  1
0.13338721  0
0.9999999  1
0.0  0
0.99999195  1
5.364418e-07  0
0.9999921  1
0.0  0
0.99999946  1
0.0  0
0.99951255  1
0.0010938346  0
0.9949441  1
0.09489617  0
0.9999989  1
0.007644981  0
0.99999154  1
0.0  0
0.9996947  1
1.54078e-05  0
0.9999994  1
3.2782555e-07  0
0.9999996  1
0.80157065  0
1.0  1
1.1920929e-07  0
1.0  1
0.0  0
0.9997449  1
0.0  0
0.9999704  1
0.0  0
0.9999999  1
0.0  0
0.99998474  1
8.6426735e-07  0
0.99999964  1
0.0  0
0.99999976  1
0.0  0
0.9999

0.9999888  1
0.0  0
0.9996083  1
2.0861626e-07  0
0.9999814  1
6.2584877e-07  0
0.9999875  1
0.00014725327  0
0.9998141  1
0.0  0
0.9995437  1
0.0  0
0.9998957  1
0.0  0
0.9999459  1
0.0  0
4.172325e-07  1
0.0  0
1.013279e-06  1
0.0  0
0.3691975  1
3.874302e-06  0
0.99828756  1
0.0  0
0.9999509  1
0.0  0
0.99936825  1
0.013083547  0
0.9999827  1
0.0  0
0.99995965  1
0.0  0
0.2079038  1
0.00015312433  0
0.6218281  1
0.0012923181  0
0.99987936  1
1.1920929e-07  0
0.99999976  1
0.050487757  0
0.9999999  1
0.0  0
0.99669003  1
0.0  0
0.97063315  1
0.00054225326  0
0.99994284  1
0.00043439865  0
0.99995637  1
0.0  0
0.99954724  1
0.0  0
0.99307775  1
0.0  0
0.99816746  1
0.010613769  0
0.9892767  1
0.0  0
0.9853691  1
3.618002e-05  0
0.99712074  1
0.0  0
0.9979399  1
4.2676926e-05  0
0.9999852  1
0.0  0
0.9999994  1
0.0  0
0.99999964  1
0.0  0
0.99999976  1
0.0  0
0.9998659  1
0.0  0
0.99931294  1
0.0  0
0.9999995  1
1.4007092e-06  0
1.0  1
0.0  0
0.99448943  1
0.0  0
0.98679215  1
9.715557

0.99688566  1
1.8775463e-06  0
0.9991789  1
7.1525574e-07  0
0.99927247  1
0.0  0
0.99964356  1
1.3113022e-06  0
0.9999287  1
0.0  0
0.9999957  1
0.0  0
0.9999963  1
1.7881393e-07  0
0.99996537  1
0.0  0
0.9999759  1
1.2218952e-06  0
0.99999684  1
0.0  0
0.9999976  1
0.0  0
0.9999832  1
0.0  0
0.9999608  1
2.0861626e-07  0
0.9999865  1
0.0  0
0.99999475  1
0.0  0
0.9999914  1
0.0  0
0.999974  1
0.0  0
0.9999881  1
5.507469e-05  0
0.9999355  1
0.0  0
0.99974513  1
0.0  0
0.99984884  1
0.0  0
0.9998842  1
0.0  0
0.9999378  1
5.632639e-06  0
0.9999349  1
2.1576881e-05  0
0.9999969  1
0.0  0
0.99997926  1
0.0  0
0.99799657  1
0.0  0
0.99962753  1
2.0861626e-07  0
0.99993277  1
8.940697e-08  0
0.9999616  1
0.0  0
0.9999585  1
0.0  0
0.99969214  1
0.0  0
0.9985923  1
0.0  0
0.9998859  1
4.1127205e-06  0
0.9999902  1
0.0  0
0.9999647  1
0.0  0
0.99999285  1
0.0  0
0.99999833  1
0.0  0
0.99987495  1
0.0  0
0.9999671  1
1.1920929e-07  0
0.99999505  1
5.9604645e-08  0
0.9999846  1
2.9802322e-08 

In [26]:
tr_y

array([1, 0, 1, ..., 0, 1, 0])

In [ ]:
# def cos_distance(vects):
#     y_true, y_pred = vects
#     y_true = K.l2_normalize(y_true, axis=-1)
#     y_pred = K.l2_normalize(y_pred, axis=-1)
#     return K.mean(1 - K.sum((y_true * y_pred), axis=-1))

# def cosine_distance(vests):
#     x, y = vests
#     x = K.l2_normalize(x, axis=-1)
#     y = K.l2_normalize(y, axis=-1)
#     return -K.mean(x * y, axis=-1)

# def cos_dist_output_shape(shapes):
#     shape1, shape2 = shapes
#     shape=list(shape1)
#     assert len(shape) == 2
#     shape[-1] = 1
#     return tuple(shape)

# output_cos = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([processed_a, processed_b])